In [1]:
# Imports

import pandas as pd
import numpy as np
from pprint import pprint
import re

In [2]:
# Import the data to dataframes

aacIn = pd.read_csv('../Data/Raw/Austin_Animal_Center_Intakes_20250419.csv')
aacOut = pd.read_csv('../Data/Raw/Austin_Animal_Center_Outcomes_20250419.csv')

Let's get the lay of the land, as they say.

In [3]:
# Headers--check them out
# First for aacIn (intakes)

aacIn.head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A521520,Nina,10/01/2013 07:51:00 AM,October 2013,Norht Ec in Austin (TX),Stray,Normal,Dog,Spayed Female,7 years,Border Terrier/Border Collie,White/Tan
1,A664235,NaN,10/01/2013 08:33:00 AM,October 2013,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
2,A664236,NaN,10/01/2013 08:33:00 AM,October 2013,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
3,A664237,NaN,10/01/2013 08:33:00 AM,October 2013,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
4,A664233,Stevie,10/01/2013 08:53:00 AM,October 2013,7405 Springtime in Austin (TX),Stray,Injured,Dog,Intact Female,3 years,Pit Bull Mix,Blue/White


In [4]:
# Then for aacOut (outcomes)

aacOut.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A882831,*Hamilton,07/01/2023 06:12:00 PM,Jul 2023,03/25/2023,Adoption,NaN,Cat,Neutered Male,3 months,Domestic Shorthair Mix,Black/White
1,A794011,Chunk,05/08/2019 06:20:00 PM,May 2019,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
2,A776359,Gizmo,07/18/2018 04:02:00 PM,Jul 2018,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
3,A821648,NaN,08/16/2020 11:38:00 AM,Aug 2020,08/16/2019,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray
4,A720371,Moose,02/13/2016 05:59:00 PM,Feb 2016,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff


In [5]:
# Check out columns

aacIn.columns

Index(['Animal ID', 'Name', 'DateTime', 'MonthYear', 'Found Location',
       'Intake Type', 'Intake Condition', 'Animal Type', 'Sex upon Intake',
       'Age upon Intake', 'Breed', 'Color'],
      dtype='object')

In [6]:
aacOut.columns

Index(['Animal ID', 'Name', 'DateTime', 'MonthYear', 'Date of Birth',
       'Outcome Type', 'Outcome Subtype', 'Animal Type', 'Sex upon Outcome',
       'Age upon Outcome', 'Breed', 'Color'],
      dtype='object')

In [7]:
print(f'The Intakes dataset shape: {aacIn.shape}')
print(f'The Outcomes dataset shape: {aacOut.shape}')

The Intakes dataset shape: (173299, 12)
The Outcomes dataset shape: (173078, 12)


In [8]:
aacIn.describe()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
count,173299,123553,173299,173299,173299,173299,173299,173299,173297,173298,173299,173299
unique,155805,29741,119399,139,70010,6,20,5,5,55,3003,660
top,A721033,Luna,09/23/2016 12:00:00 PM,June 2015,Austin (TX),Stray,Normal,Dog,Intact Male,1 year,Domestic Shorthair Mix,Black/White
freq,33,760,64,2189,31467,118800,146790,94428,58825,28210,33662,17930


In [9]:
aacOut.describe()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
count,173078,123540,173078,173078,173078,173032,79301,173078,173076,173069,173078,173078
unique,155605,29688,144670,139,8657,11,26,5,5,55,3002,662
top,A721033,Luna,04/18/2016 12:00:00 AM,Jun 2019,05/01/2016,Adoption,Partner,Dog,Neutered Male,1 year,Domestic Shorthair Mix,Black/White
freq,33,756,39,2244,121,84226,40250,94200,60693,28670,34037,17930


### Checking Duplicates

In [10]:
# How many dupes do we have?

in_dupes = aacIn.duplicated(subset = ['Animal ID', 'DateTime', 'Animal Type', 'Name']).sum()
out_dupes = aacOut.duplicated(subset = ['Animal ID', 'DateTime', 'Animal Type', 'Name']).sum()

print(f'Duplicates in Intakes: {in_dupes}')
print(f'Duplicates in Outcomes: {out_dupes}')

Duplicates in Intakes: 42
Duplicates in Outcomes: 26


There are very few duplicates, which is good. I think if the ID, date, name, and animal type match, they are reasonably full duplicates and can be dropped.

In [11]:
# Let's drop them

aacIn.drop_duplicates(subset=['Animal ID', 'DateTime', 'Animal Type', 'Name'], keep='first', inplace=True)
aacOut.drop_duplicates(subset=['Animal ID', 'DateTime', 'Animal Type', 'Name'], keep='first', inplace=True)

In [12]:
# Check it worked

in_new_dupes = aacIn.duplicated(subset = ['Animal ID', 'DateTime', 'Animal Type', 'Name']).sum()
out_new_dupes = aacOut.duplicated(subset = ['Animal ID', 'DateTime', 'Animal Type', 'Name']).sum()

print(f'Duplicates remaining in Intakes: {in_new_dupes}')
print(f'Duplicates remaining in Outcomes: {out_new_dupes}')

Duplicates remaining in Intakes: 0
Duplicates remaining in Outcomes: 0


### Animal ID

In [13]:
# Check out 'Animal ID' column for joining purposes

# Grabbing some variables to print
ids_in = len(aacIn['Animal ID'].unique())
ids_out = len(aacOut['Animal ID'].unique())

print(f'Number of unique Animal IDs in Intakes: {ids_in}')
print(f'Number of rows in aacIn: {aacIn.shape[0]}\n')

print(f'Number of unique Animal IDs in Outcomes: {ids_out}')
print(f'Number of rows in aacOut: {aacOut.shape[0]}\n')

Number of unique Animal IDs in Intakes: 155805
Number of rows in aacIn: 173257

Number of unique Animal IDs in Outcomes: 155605
Number of rows in aacOut: 173052



There are fewer Animal IDs than rows in both datasets, and both datasets have different numbers of rows and IDs; this is likely because many of the animals from Intakes have not received an Outcome yet.

In [14]:
# There are fewer Animal IDs than rows in both datasets, and both datasets have different numbers of rows and IDs

print(f'There are {ids_in - ids_out} more unique Animal IDs in Intakes than Outtakes')

There are 200 more unique Animal IDs in Intakes than Outtakes


200 more Animal IDs in aacIn than aacOut makes sense to me (it would indicate that there are 200 animals currently in the shelter).

In [15]:
# How many IDs are missing from each dataset, if any?

# Variables to print and reference

id_missing_in = aacIn['Animal ID'].isnull().sum()
id_missing_out = aacOut['Animal ID'].isnull().sum()

duped_ids_in = aacIn.duplicated(subset = 'Animal ID').sum()
duped_ids_out = aacOut.duplicated(subset = 'Animal ID').sum()

# Prints

print(f'There are {id_missing_in} missing IDs in Intakes.')
print(f'There are {id_missing_out} missing IDs in Outcomes.\n')
print(f'There are {duped_ids_in} IDs duplicated in Intakes')
print(f'There are {duped_ids_out} IDs duplicated in Outcomes')

There are 0 missing IDs in Intakes.
There are 0 missing IDs in Outcomes.

There are 17452 IDs duplicated in Intakes
There are 17447 IDs duplicated in Outcomes


In [16]:
# Print rows with duplicated IDs

# Intakes

aacIn[aacIn.duplicated(subset = 'Animal ID', keep = False)].sort_values('Animal ID').head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
22374,A006100,Scamp,12/19/2014 10:21:00 AM,December 2014,8700 Research Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White
6926,A006100,Scamp,03/07/2014 02:26:00 PM,March 2014,8700 Research in Austin (TX),Public Assist,Normal,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White
75925,A006100,Scamp,12/07/2017 02:07:00 PM,December 2017,Colony Creek And Hunters Trace in Austin (TX),Stray,Normal,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White
13576,A245945,Boomer,07/03/2014 05:55:00 PM,July 2014,Garden And Mildred in Austin (TX),Stray,Normal,Dog,Neutered Male,14 years,Labrador Retriever Mix,Tan
29344,A245945,Boomer,05/20/2015 10:34:00 PM,May 2015,7403 Blessing Ave in Austin (TX),Stray,Normal,Dog,Neutered Male,15 years,Labrador Retriever Mix,Tan


In [17]:
# Outcomes

aacOut[aacOut.duplicated(subset = 'Animal ID', keep = False)].sort_values('Animal ID').head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
150493,A006100,Scamp,03/08/2014 05:10:00 PM,Mar 2014,07/09/2007,Return to Owner,NaN,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White
132575,A006100,Scamp,12/07/2017 12:00:00 AM,Dec 2017,07/09/2007,Return to Owner,NaN,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White
74205,A006100,Scamp,12/20/2014 04:35:00 PM,Dec 2014,07/09/2007,Return to Owner,NaN,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White
118420,A245945,Boomer,07/04/2014 03:26:00 PM,Jul 2014,05/23/2000,Return to Owner,NaN,Dog,Neutered Male,14 years,Labrador Retriever Mix,Tan
90873,A245945,Boomer,05/25/2015 11:49:00 AM,May 2015,05/23/2000,Transfer,Partner,Dog,Neutered Male,15 years,Labrador Retriever Mix,Tan


It appears that duplicated Animal IDs are likely the result of the same animal coming into or out of the shelter multiple times. To deal with this, I think I will have to do a merge connecting Intake and Outcome dates as closely as possible for each animal.

In [18]:
# Strip white spaces, in case there are any

aacIn['Animal ID'] = aacIn['Animal ID'].str.strip()
aacOut['Animal ID'] = aacOut['Animal ID'].str.strip()

### Column Names

In [19]:
# I will standardize the style of column names for convenience and ease of use

aacIn = aacIn.rename(columns = {"Animal ID" : "AnimalID", "DateTime" : "IntakeDT", "Found Location" : "FoundLocation", "Intake Type" : "IntakeType", 
                                "Intake Condition" : "IntakeCondition", "Animal Type" : "AnimalType", "Sex upon Intake" : "IntakeSex",
                                "Age upon Intake" : "IntakeAge"})
aacOut = aacOut.rename(columns = {"Animal ID" : "AnimalID", "DateTime" : "OutcomeDT", "Date of Birth" : "BirthDate", "Outcome Type" : "OutcomeType",
                                  "Outcome Subtype" : "OutcomeSubtype", "Animal Type" : "AnimalType", "Sex upon Outcome" : "OutcomeSex", 
                                  "Age upon Outcome" : "OutcomeAge"})

### Nulls

In [20]:
# Check for nulls

print(f'Nulls in the Intakes dataset:\n', aacIn.isnull().sum(), sep = "")
print()
print(f'Nulls in the Outcomes dataset:\n', aacOut.isnull().sum(), sep = "")

Nulls in the Intakes dataset:
AnimalID               0
Name               49740
IntakeDT               0
MonthYear              0
FoundLocation          0
IntakeType             0
IntakeCondition        0
AnimalType             0
IntakeSex              2
IntakeAge              1
Breed                  0
Color                  0
dtype: int64

Nulls in the Outcomes dataset:
AnimalID              0
Name              49533
OutcomeDT             0
MonthYear             0
BirthDate             0
OutcomeType          46
OutcomeSubtype    93762
AnimalType            0
OutcomeSex            2
OutcomeAge            9
Breed                 0
Color                 0
dtype: int64


Given that most of the nulls are in Names and columns that are independent between datasets, I think I can deal with nulls after the join.

### Datatypes

In [21]:
print('Intakes:\n', aacIn.dtypes, sep = "")
print()
print('Outcomes:\n', aacOut.dtypes, sep = "")

Intakes:
AnimalID           object
Name               object
IntakeDT           object
MonthYear          object
FoundLocation      object
IntakeType         object
IntakeCondition    object
AnimalType         object
IntakeSex          object
IntakeAge          object
Breed              object
Color              object
dtype: object

Outcomes:
AnimalID          object
Name              object
OutcomeDT         object
MonthYear         object
BirthDate         object
OutcomeType       object
OutcomeSubtype    object
AnimalType        object
OutcomeSex        object
OutcomeAge        object
Breed             object
Color             object
dtype: object


Most of these columns are and should be objects, but DateTime and MonthYear should be DateTime objects, and age should likely be an integer or float column. Eventually, we may want to codify some of these categories, particularly the Sex and Animal Type columns.

### Datetimes

In [22]:
# Transform IntakeDT and OutcomeDT to datetimes

aacIn['IntakeDT'] = pd.to_datetime(aacIn['IntakeDT'], format = "%m/%d/%Y %I:%M:%S %p")
aacOut['OutcomeDT'] = pd.to_datetime(aacOut['OutcomeDT'], format = "%m/%d/%Y %I:%M:%S %p")

# Check that it worked

print(f'IntakeDate is now a {aacIn['IntakeDT'].dtype} datatype.')
print(f'OutcomeDate is now a {aacOut['OutcomeDT'].dtype} datatype.')

IntakeDate is now a datetime64[ns] datatype.
OutcomeDate is now a datetime64[ns] datatype.


In [23]:
# I don't really need the "MonthYear" column, as I can just extract the month and year now from those datetime values

aacIn = aacIn.drop(columns = 'MonthYear')
aacOut = aacOut.drop(columns = 'MonthYear')

# Check that it did indeed work...

print(aacIn.columns)
print(aacOut.columns)

Index(['AnimalID', 'Name', 'IntakeDT', 'FoundLocation', 'IntakeType',
       'IntakeCondition', 'AnimalType', 'IntakeSex', 'IntakeAge', 'Breed',
       'Color'],
      dtype='object')
Index(['AnimalID', 'Name', 'OutcomeDT', 'BirthDate', 'OutcomeType',
       'OutcomeSubtype', 'AnimalType', 'OutcomeSex', 'OutcomeAge', 'Breed',
       'Color'],
      dtype='object')


## More Column Examination

#### Independent (non-duplicated) columns

### IntakeType

In [24]:
print(aacIn['IntakeType'].value_counts())

IntakeType
Stray                 118785
Owner Surrender        35453
Public Assist          10392
Wildlife                6463
Abandoned               1900
Euthanasia Request       264
Name: count, dtype: int64


There are only 6 categories, with no mistypes or significant outliers. I think these are pretty good categories to work with.

### IntakeCondition

In [25]:
print(aacIn['IntakeCondition'].value_counts())

IntakeCondition
Normal        146754
Injured        10755
Sick            7920
Nursing         3878
Neonatal        1917
Medical          596
Aged             524
Other            353
Pregnant         155
Feral            144
Med Attn          86
Behavior          79
Unknown           46
Med Urgent        20
Neurologic        13
Parvo              7
Space              4
Agonal             4
Panleuk            1
Congenital         1
Name: count, dtype: int64


These I think I can definitely consolidate, to get rid of the outliers with just a few entries and group categories that represent similar conditions together for clarity and simplicity.

In [26]:
# Dictionary to map original intake conditions to condensed categories: 
# Normal, Injured, Sick, Medical, Neonatal, Nursing, Pregnant, Aged, Behavior, Feral, Other

conditions = {
    "Normal": "Normal", # Including specifying those columns I would like to leave as-is
    "Injured": "Injured",
    "Sick": "Sick",
    "Medical": "Medical", 
    "Med Attn": "Medical",
    "Med Urgent": "Medical",
    "Neurologic": "Medical", # Likely refers to neurological conditions, which will fall under Medical
    "Parvo": "Medical", # Refers to an illness especially common among puppies; I'm actually surprised there were so few in this category
    "Panleuk": "Medical", # Panleukemia, also a medical illness
    "Agonal": "Medical", # Refers to a dying creature; it is a severe condition, but there were only 4 in this category, I will change it to medical
    "Congenital": "Medical", # Likely refers to a genetic condition or disability--medical
    "Neonatal": "Neonatal",
    "Nursing": "Nursing",
    "Pregnant": "Pregnant",
    "Aged": "Aged",
    "Behavior": "Behavior",
    "Feral": "Feral",
    "Other": "Other",
    "Unknown": "Other",
    "Space": "Other" # I believe this refers to an animal being transferred between shelters relating to lack of space; 
                        # why it is here, I do not know
}

# Apply mapping to the 'Intake Condition' column

aacIn['IntakeCondition'] = aacIn['IntakeCondition'].map(conditions)

In [27]:
# Check that this worked

aacIn['IntakeCondition'].value_counts()

IntakeCondition
Normal      146754
Injured      10755
Sick          7920
Nursing       3878
Neonatal      1917
Medical        728
Aged           524
Other          403
Pregnant       155
Feral          144
Behavior        79
Name: count, dtype: int64

Seems to have worked perfectly!

### FoundLocation

Despite being extra information and possibly interesting to evaluate, this column is just not within the scope of this project; so, we will drop it.

In [28]:
# Drop

aacIn = aacIn.drop(columns = 'FoundLocation')

In [29]:
# Check, as always

aacIn.columns

Index(['AnimalID', 'Name', 'IntakeDT', 'IntakeType', 'IntakeCondition',
       'AnimalType', 'IntakeSex', 'IntakeAge', 'Breed', 'Color'],
      dtype='object')

### OutcomeType

I suspect this will be similar to IntakeType; let's take a look

In [30]:
print(aacOut['OutcomeType'].value_counts())

OutcomeType
Adoption           84214
Transfer           48499
Return to Owner    25629
Euthanasia         10766
Died                1663
Rto-Adopt           1236
Disposal             872
Missing               92
Relocate              28
Stolen                 5
Lost                   2
Name: count, dtype: int64


A few of these stand out. "Rto-Adopt" seems odd, since "Return to Owner" is already a category, and "Disposal" is...odd, considering "Euthanasia" and "Died" are both already categories of their own. My suspicion is that it refers to Euthanasia indirectly. "Relocate" seems very similar to "Transfer", and the other three ("Missing", "Stolen", and "Lost") can likely be grouped into one category.

In [31]:
# First, combine "Rto-Adopt" and "Return to Owner"

aacOut.loc[(aacOut['OutcomeType'] == 'Rto-Adopt'), 'OutcomeType'] = 'Return to Owner'

# Check

aacOut['OutcomeType'].value_counts()

OutcomeType
Adoption           84214
Transfer           48499
Return to Owner    26865
Euthanasia         10766
Died                1663
Disposal             872
Missing               92
Relocate              28
Stolen                 5
Lost                   2
Name: count, dtype: int64

I am uncertain about "Disposal," though I expect it to be an alternate wording for "Euthanasia." I will check by first evaluating the timestamps for the two labels (the thought behind this is that possibly the wording could have changed for or at some point in time). If the times do not greatly overlap, it is almost certainly just a rewording.

In [32]:
# Grab the year values for each as variables

disposal_years = aacOut.loc[(aacOut['OutcomeType'] == 'Disposal'), 'OutcomeDT']
euth_years = aacOut.loc[(aacOut['OutcomeType'] == 'Euthanasia'), 'OutcomeDT']

# What years do each category fall within?

print(f'"Disposal" was used during the years: {sorted(disposal_years.dt.year.unique())}')
print(f'"Euthanasia" was used during the years: {sorted(euth_years.dt.year.unique())}')

"Disposal" was used during the years: [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
"Euthanasia" was used during the years: [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]


Both have been used throughout the dataset, it appears. I will investigate OutcomeSubtype for the answer to this question.

In [33]:
print(aacOut['OutcomeSubtype'].value_counts())

OutcomeSubtype
Partner                40247
Foster                 17851
Rabies Risk             4891
Suffering               4123
Snr                     4078
SCRP                    3208
Out State                931
In Kennel                868
Aggressive               612
Offsite                  512
In Foster                416
Medical                  353
At Vet                   352
Field                    242
Behavior                 176
Enroute                  112
Court/Investigation      100
Emergency                 53
Underage                  37
In Surgery                33
Customer S                20
Prc                       20
Possible Theft            16
Barn                      16
In State                  12
Emer                      11
Name: count, dtype: int64


In [34]:
# Lots of options there. Let's see which ones correspond with "Euthanasia" and "Disposal"

# Which correspond with euthanasia?

print(aacOut.loc[(aacOut['OutcomeType'] == 'Euthanasia'), 'OutcomeSubtype'].value_counts())

OutcomeSubtype
Rabies Risk            4839
Suffering              4123
Aggressive              612
Medical                 353
At Vet                  240
Behavior                176
Court/Investigation     100
Underage                 37
Name: count, dtype: int64


In [35]:
# And which correspond with disposal?

print(aacOut.loc[(aacOut['OutcomeType'] == 'Disposal'), 'OutcomeSubtype'].value_counts())

OutcomeSubtype
Rabies Risk    52
Name: count, dtype: int64


There we go! "Rabies Risk" appears in both groups (in fact, it is the only one to appear with disposal), and is a pretty obvious cause for euthanasia. I'd say it's safe to consider "Disposal" a subset of "Euthanasia". Let's combine the two.

In [36]:
# Map "Disposal" to "Euthanasia"

aacOut.loc[(aacOut['OutcomeType'] == 'Disposal'), 'OutcomeType'] = 'Euthanasia'

# Check our work, as always

print(aacOut['OutcomeType'].value_counts())

OutcomeType
Adoption           84214
Transfer           48499
Return to Owner    26865
Euthanasia         11638
Died                1663
Missing               92
Relocate              28
Stolen                 5
Lost                   2
Name: count, dtype: int64


In [37]:
# "Relocate" seems awfully similar to "Transfer". Let's look at Subtypes again to make sure.

print('The "Relocate" Outcome Subtypes are ' + str(aacOut.loc[(aacOut['OutcomeType'] == 'Relocate'), 'OutcomeSubtype'].unique()))
print('The "Transfer" Outcome Subtypes are ' + str(aacOut.loc[(aacOut['OutcomeType'] == 'Transfer'), 'OutcomeSubtype'].unique()))

The "Relocate" Outcome Subtypes are [nan]
The "Transfer" Outcome Subtypes are ['Partner' 'Snr' 'SCRP' 'Out State' 'Barn' 'In State' 'Emer']


That wasn't as helpful as I had hoped, but I do think that "Relocate" should be combined with "Transfer".

In [38]:
# Combine "Relocate" to "Transfer"

aacOut.loc[(aacOut['OutcomeType'] == 'Relocate'), 'OutcomeType'] = 'Transfer'

# Check

print(aacOut['OutcomeType'].value_counts())

OutcomeType
Adoption           84214
Transfer           48527
Return to Owner    26865
Euthanasia         11638
Died                1663
Missing               92
Stolen                 5
Lost                   2
Name: count, dtype: int64


In [39]:
# Finally, compiling the last 3 into "Missing"

aacOut.loc[(aacOut['OutcomeType'] == 'Stolen'), 'OutcomeType'] = 'Missing'
aacOut.loc[(aacOut['OutcomeType'] == 'Lost'), 'OutcomeType'] = 'Missing'

# Check, yet again

print(aacOut['OutcomeType'].value_counts())

OutcomeType
Adoption           84214
Transfer           48527
Return to Owner    26865
Euthanasia         11638
Died                1663
Missing               99
Name: count, dtype: int64


I think that's it for the OutcomeTypes! There are 6 remaining categories, and they feel to me distinct and specific. On to Subtypes!

### OutcomeSubtype

In [40]:
# What are we looking at?

print(aacOut['OutcomeSubtype'].value_counts())

OutcomeSubtype
Partner                40247
Foster                 17851
Rabies Risk             4891
Suffering               4123
Snr                     4078
SCRP                    3208
Out State                931
In Kennel                868
Aggressive               612
Offsite                  512
In Foster                416
Medical                  353
At Vet                   352
Field                    242
Behavior                 176
Enroute                  112
Court/Investigation      100
Emergency                 53
Underage                  37
In Surgery                33
Customer S                20
Prc                       20
Possible Theft            16
Barn                      16
In State                  12
Emer                      11
Name: count, dtype: int64


First, I remap Subtypes that are obviously abbreviations or similar, using common shelter industry terms.

In [41]:
# Snr -> Shelter Neuter Return (for clarity)

aacOut.loc[(aacOut['OutcomeSubtype'] == 'Snr'), 'OutcomeSubtype'] = 'Shelter Neuter Return'

# SCRP -> Scratched Person

aacOut.loc[(aacOut['OutcomeSubtype'] == 'SCRP'), 'OutcomeSubtype'] = 'Scratched Person'

# Emer -> Emergency

aacOut.loc[(aacOut['OutcomeSubtype'] == 'Emer'), 'OutcomeSubtype'] = 'Emergency'

# Out State -> Out of State

aacOut.loc[(aacOut['OutcomeSubtype'] == 'Out State'), 'OutcomeSubtype'] = 'Out of State'

# Prc -> Pet Resource Center

aacOut.loc[(aacOut['OutcomeSubtype'] == 'Prc'), 'OutcomeSubtype'] = 'Pet Resource Center'

In [42]:
# Check that everything is in order and worked

print(aacOut['OutcomeSubtype'].value_counts())

OutcomeSubtype
Partner                  40247
Foster                   17851
Rabies Risk               4891
Suffering                 4123
Shelter Neuter Return     4078
Scratched Person          3208
Out of State               931
In Kennel                  868
Aggressive                 612
Offsite                    512
In Foster                  416
Medical                    353
At Vet                     352
Field                      242
Behavior                   176
Enroute                    112
Court/Investigation        100
Emergency                   64
Underage                    37
In Surgery                  33
Customer S                  20
Pet Resource Center         20
Possible Theft              16
Barn                        16
In State                    12
Name: count, dtype: int64


Now I will explore some of the less clear ones to double check that they have been translated correctly and make sure that they do indeed warrant their own category.

In [43]:
# Scratched Person subtype

print(aacOut.loc[(aacOut['OutcomeSubtype'] == "Scratched Person"), 'OutcomeType'].unique())

['Transfer']


Since Austin is an almost-entirely kill-free city, transferring animals that normally would be euthanized probably makes the most sense as a second resort. I think this is likely the right interpretation of "SCRP".

In [44]:
# Out of State subtype

print(aacOut.loc[(aacOut['OutcomeSubtype'] == "Out of State"), 'OutcomeType'].unique())

['Transfer']


That subtype refers only to the Transfer outcome, which makes sense.

In [45]:
# What does "In Kennel" refer to?

print(aacOut.loc[(aacOut['OutcomeSubtype'] == "In Kennel"), 'OutcomeType'].unique())

['Died' 'Missing']


I see why they would want to keep track of these events happening within the kennel, but it doesn't particularly help us get to our goal.

In [46]:
# Aggressive?

print(aacOut.loc[(aacOut['OutcomeSubtype'] == "Aggressive"), 'OutcomeType'].unique())

['Euthanasia']


In [47]:
# Enroute?

print(aacOut.loc[(aacOut['OutcomeSubtype'] == "Enroute"), 'OutcomeType'].unique())

['Died' 'Missing']


Same deal as "In Kennel". It makes sense that this was kept track of, but I'm not sure how helpful it will be.

In [48]:
# Field

print(aacOut.loc[(aacOut['OutcomeSubtype'] == "Field"), 'OutcomeType'].unique())

['Return to Owner']


In [49]:
# Change that to "In Field" for my own sake

aacOut.loc[(aacOut['OutcomeSubtype'] == 'Field'), 'OutcomeSubtype'] = 'In Field'

In [50]:
# What is "Customer S"?

print(aacOut.loc[(aacOut['OutcomeSubtype'] == "Customer S"), 'OutcomeType'].unique())

['Return to Owner']


That clarified nothing. I think I'll leave it for now and investigate it some more later. "Customer Service," perhaps? Or "Customer Support?"

In [51]:
# I suspect "Barn" refers to barn cats or a barn cat program. Let's see.

print(aacOut.loc[(aacOut['OutcomeSubtype'] == "Barn"), 'OutcomeType'].unique())
print(aacOut.loc[(aacOut['OutcomeSubtype'] == "Barn"), 'AnimalType'].value_counts())

['Transfer' 'Adoption']
AnimalType
Cat    15
Dog     1
Name: count, dtype: int64


Likely but again not fully clear; I will leave this for EDA research.

I'm going to be leaving the rest of this column alone, since the categories are quite specific and circumstantial.

There are a significant number of null values in this column; it may eventually be dropped, but for now I will leave it alone.

### BirthDate

In [52]:
# Take a look

print(aacOut['BirthDate'].value_counts())

BirthDate
05/01/2016    121
05/05/2014    120
04/21/2014    117
09/01/2015    117
04/20/2015    108
             ... 
05/10/2004      1
08/03/1998      1
10/23/2001      1
04/08/2001      1
01/16/2025      1
Name: count, Length: 8657, dtype: int64


In [53]:
# What does this look like in the dataset?

print(aacOut[['AnimalID', 'Name', 'BirthDate']].head())

  AnimalID       Name   BirthDate
0  A882831  *Hamilton  03/25/2023
1  A794011      Chunk  05/02/2017
2  A776359      Gizmo  07/12/2017
3  A821648        NaN  08/16/2019
4  A720371      Moose  10/08/2015


So this actually does comprise of dates, which means I probably will leave it alone. I suspect I will need to change the datatype though...

In [54]:
print(f'"BirthDate" is a(n) {aacOut['BirthDate'].dtypes}.')

"BirthDate" is a(n) object.


In [55]:
# Convert to a datetime object

aacOut['BirthDate'] = pd.to_datetime(aacOut['BirthDate'], format = "%m/%d/%Y")

In [56]:
# Check our work, as always

print(f'"BirthDate" is a(n) {aacOut['BirthDate'].dtypes}.')

"BirthDate" is a(n) datetime64[ns].


In [57]:
aacOut['BirthDate'].head()

0   2023-03-25
1   2017-05-02
2   2017-07-12
3   2019-08-16
4   2015-10-08
Name: BirthDate, dtype: datetime64[ns]

## Columns Duplicated Between Datasets

### Name

#### Intakes

In [58]:
# How many names are we looking at?

print(len(aacIn['Name'].unique()))

29742


In [59]:
# That's not quite 170,000. Let's see how many are just unnamed

print(aacIn['Name'].isnull().sum())

49740


In [60]:
# Still not quite adding up. How many are duplicated?

print(aacIn['Name'].duplicated().sum())

143515


In [61]:
# Uh-huh. I suppose that's where they all are.
# Duplicated names are fine, though, since we're not using this as a merge column or for any real analysis.
# Let's preview some names

names_full = aacIn['Name'].unique().tolist()
for name in names_full:
    print(name)

Nina
nan
Stevie
*Donnie
Pippin
Sylvio
Claire
Heather
*Petunia
*Bismark
*Mason
*Dixon
*Asti
*Kochka
*Grace
Buffy
Cate
Moose
Estrella
Lucio
Sera
Mcdermot
*Pogo
Coco
Jade
Otter
Juliet
Sugar
Daisy
*Trevor
*Romeo
*Juliet
*Paris
*Mercutio
*Tybalt
*Benvolio
*Taylor
*Tyler
*Doc
*Happy
Neville
Annabelle
*Tiny
*Sneezy
Bruce
*Romo
Lizzy
Chan
Zoe
Suzie Q
Honey
Boris
*Eloise
Ned
*Wilbur
Zuzu
Angie
*J
*Trey
Butter
Lillian
Zooe
*Miso
*Elvira
*Frisco
Elvis
Tazz
Ziggy
Jewels
Azul
Frosty
*Lula Belle
Deeohhgee
Cowboy
Gotti
Zeus
*Halloween
October
*Birch
Chevy
*Aries
*Poodle
Raisin
Miley
Chewey
Serenity
Walter
Amie
*Harry
*Beth
*Daisy
Simba
Canelo
Spot
Missy
Batty
*Sunshine
Wyatts
*Candy Corn
Sebastian
*Tudor
Brownie
*Riley
Roxie
Pumpkin
Baby
Jumper
Diesle
Poe
Miss Kitty
Carlos
*Edgar
Raven
Bandit
*Jesse
Bosco
*Broadway
*Arthur
Charley
Costello
*Peaches
*Chris Angel
Nikole
*Nikko
*Hercules
*Teddy
Dahlia
Charcoal
Gypsy
*Flower
T-Che
Mittens
Matilda
Pearl
*Dancer
Travis
*Allie
*Gizzmo
*Franklin
*Sarah
Zane


I've noticed lots of asterisks (*), and am wary also of things such as leading and trailing spaces, so let's remove those


In [62]:
# I will keep most punctuation, for names like Mr. Dog or O'Malley

aacIn['Name'] = aacIn['Name'].astype(str)
aacIn['Name'] = aacIn['Name'].apply(lambda x: re.sub(r"[^\w\s\.\'\-]", '', x).strip())

In [63]:
# Check if that worked...

names_new = aacIn['Name'].unique().tolist()
for name in names_new[0:20]:
    print(name)

Nina
nan
Stevie
Donnie
Pippin
Sylvio
Claire
Heather
Petunia
Bismark
Mason
Dixon
Asti
Kochka
Grace
Buffy
Cate
Moose
Estrella
Lucio


In [64]:
# Which names are most common?

print(aacIn['Name'].value_counts()[aacIn['Name'].value_counts() > 200])

Name
nan         49740
Luna          812
Max           792
Bella         723
Daisy         575
Charlie       550
Rocky         528
Coco          463
Lucy          456
Blue          438
Buddy         429
Lola          417
Princess      417
Milo          338
Oreo          336
Zeus          333
Jack          328
Cookie        307
Shadow        304
Bear          295
Lily          290
Molly         289
Sadie         283
Toby          279
Lucky         271
Mia           267
Duke          261
Pepper        260
Nala          259
Ginger        246
Chloe         241
Leo           239
Penny         234
Chico         232
Oliver        232
Rosie         227
Maggie        227
King          224
Ruby          220
Lady          216
Honey         216
Bailey        215
Smokey        212
Stella        211
Marley        205
Name: count, dtype: int64


I think that will do, for now. Names likely will not be a much of an analytical aspect of the project; they are just fun ways to show the stories of these animals when it comes to my finishing presentation and write-up.

#### Outcomes

In [65]:
# Doing the same thing as above with Outcome names: stripping white space, asterisks, and other unusual characters, while leaving certain punctuation

aacOut['Name'] = aacOut['Name'].astype(str)
aacOut['Name'] = aacOut['Name'].apply(lambda x: re.sub(r"[^\w\s\.\'\-]", '', x).strip())

In [66]:
# And previewing the top values

print(aacOut['Name'].value_counts()[aacOut['Name'].value_counts() > 200])

Name
nan         49533
Luna          810
Max           791
Bella         726
Daisy         574
Charlie       554
Rocky         533
Coco          464
Lucy          456
Blue          438
Buddy         426
Princess      421
Lola          415
Oreo          336
Milo          334
Zeus          333
Jack          326
Cookie        310
Shadow        303
Bear          294
Molly         293
Lily          289
Sadie         285
Toby          281
Lucky         271
Mia           267
Nala          261
Duke          260
Pepper        257
Ginger        247
Chloe         241
Leo           239
Oliver        232
Penny         232
Chico         232
Maggie        230
Rosie         227
King          222
Ruby          221
Bailey        216
Lady          214
Honey         214
Smokey        211
Stella        211
Marley        205
Name: count, dtype: int64


I'm happy to leave the names alone, otherwise. Moving on!

### AnimalType

#### Intakes

In [67]:
# Check out our categories

print(aacIn['AnimalType'].value_counts())

AnimalType
Dog          94406
Cat          69009
Other         8933
Bird           876
Livestock       33
Name: count, dtype: int64


Those categories are probably fine, although I wonder about "Other" and would love to add in a "Wildlife" category, if possible.

In [68]:
# What breeds is "Other" comprised of?

other_breeds = aacIn.loc[aacIn['AnimalType'] == 'Other', 'Breed'].unique()
pprint(other_breeds)

array(['Bat Mix', 'Rabbit Sh Mix', 'Raccoon Mix', 'Skunk', 'Raccoon',
       'Opossum Mix', 'Bat', 'Ferret', 'Californian Mix', 'Skunk Mix',
       'Rabbit Sh', 'Fox Mix', 'Opossum', 'Armadillo Mix', 'Squirrel Mix',
       'Guinea Pig Mix', 'Rabbit Sh/Lop-Mini', 'Fox', 'Turtle Mix',
       'Rex Mix', 'Guinea Pig', 'Squirrel', 'Rat', 'Rat Mix',
       'Californian', 'Coyote Mix', 'Hamster Mix', 'English Spot Mix',
       'Lionhead Mix', 'Ferret Mix', 'Silver Mix', 'Tortoise Mix',
       'Sugar Glider', 'Dutch Mix', 'Jersey Wooly Mix', 'Cottontail Mix',
       'Lop-English Mix', 'Armadillo', 'Cold Water', 'American Mix',
       'Lop-Holland Mix', 'Lionhead', 'Rabbit Lh Mix', 'Snake',
       'Dutch/Rabbit Sh', 'Snake Mix', 'Lizard Mix', 'American Sable',
       'New Zealand Wht Mix', 'Lop-Mini Mix', 'Mouse Mix', 'Hotot Mix',
       'Hamster', 'Cinnamon', 'Rabbit Lh', 'American Sable Mix',
       'Gerbil Mix', 'Angora-French Mix', 'Dutch', 'Tarantula', 'Turtle',
       'Frog', 'Snake/Snake

In [69]:
# How many breeds is that?

print(aacIn.loc[aacIn['AnimalType'] == 'Other', 'Breed'].nunique())

139


In [70]:
# With only 100-some breed examples, I think I can do this manually and create a list of all wildlife "breeds" that have shown up thus far

wildlife = ['Bat Mix', 'Raccoon Mix', 'Skunk', 'Raccoon', 'Opossum Mix', 'Bat', 'Skunk Mix', 'Fox Mix', 
            'Opossum', 'Armadillo Mix', 'Squirrel Mix', 'Fox', 'Turtle Mix', 'Squirrel', 'Rat', 'Rat Mix', 
            'Coyote Mix', 'Tortoise Mix', 'Cottontail Mix', 'Armadillo', 'Cold Water', 'Snake', 'Snake Mix', 
            'Lizard Mix', 'Mouse Mix', 'Tarantula', 'Turtle', 'Frog', 'Snake/Snake', 'Ringtail Mix', 
            'Lizard', 'Coyote', 'Bobcat Mix', 'Otter Mix', 'Tortoise', 'Deer Mix', 'Cold Water Mix', 
            'Prairie Dog Mix', 'Mouse', 'Otter', 'Cottontail', 'Bat/Mex Free-Tail', 
            'Turtle/Redeared Slider', 'Ringtail', 'Lizard/Gecko', 'Deer']

# There are some that I suspect could be wildlife, but am not fully certain of, so I will check those out

suspects = ['Turtle Mix', 'Rat', 'Rat Mix', 'Coyote Mix', 'Tortoise Mix', 'Cottontail Mix', 'Cold Water', 'Snake', 'Snake Mix',
            'Lizard Mix', 'Mouse Mix', 'Tarantula', 'Turtle', 'Frog', 'Snake/Snake', 'Lizard', 'Tortoise',
            'Turtle/Redeared Slider', 'Lizard/Gecko']

aacIn.loc[aacIn["Breed"].isin(suspects), ['Name', 'IntakeType', 'Breed', 'Color']]

,Name,IntakeType,Breed,Color
2941,nan,Wildlife,Turtle Mix,Brown/Yellow
4468,nan,Owner Surrender,Rat,Tan/White
4469,nan,Owner Surrender,Rat,Black/White
4705,nan,Stray,Rat Mix,White
5476,nan,Wildlife,Coyote Mix,Red
...,...,...,...,...
172292,nan,Public Assist,Lizard,Yellow/Black
172293,nan,Public Assist,Lizard,Yellow/Black
172633,nan,Stray,Turtle,Brown/Orange
172993,nan,Public Assist,Tarantula,Brown


In [71]:
# Hm, 363 is a low of rows to go through manually. To narrow it down, I will find which entries have a "Wildlife" intake type first

suspects_df = aacIn.loc[
    (aacIn["Breed"].isin(suspects)) &
    (aacIn["IntakeType"] == "Wildlife")
]

# Show the matching breeds

pprint(list(suspects_df["Breed"].unique()))

['Turtle Mix',
 'Coyote Mix',
 'Tortoise Mix',
 'Mouse Mix',
 'Cottontail Mix',
 'Snake',
 'Rat Mix',
 'Rat',
 'Snake Mix',
 'Turtle',
 'Tortoise',
 'Lizard']


In [72]:
# Add those names to our wildlife "breed" list

wildlife.extend(suspects_df["Breed"].unique().tolist())

In [73]:
# Now for the rest, breeds that I will need to investigate manually.

remaining_suspects = [breed for breed in suspects if breed not in suspects_df["Breed"].unique()]

# Let's print those rows

aacIn.loc[aacIn["Breed"].isin(remaining_suspects), ['Name', 'IntakeType', 'Breed', 'Color']]

,Name,IntakeType,Breed,Color
11703,Plecostomus,Public Assist,Cold Water,Black/Gray
11704,Oscar,Public Assist,Cold Water,Gray/Orange
11705,Oscar,Public Assist,Cold Water,White/Orange
11708,Koi,Public Assist,Cold Water,Orange/White
11709,Plecostomus,Public Assist,Cold Water,Black/Gray
15294,nan,Stray,Lizard Mix,Tan
24402,nan,Public Assist,Lizard Mix,Tan
26912,nan,Public Assist,Lizard Mix,Black/Yellow
32482,nan,Owner Surrender,Tarantula,Brown/Red
33074,nan,Public Assist,Frog,Brown


In [74]:
# 61 isn't too bad. Let's make a mini dataframe of these to work with.

sus_manual = aacIn.loc[aacIn["Breed"].isin(remaining_suspects), ['Name', 'IntakeType', 'Breed', 'Color']]
print(sus_manual['Breed'].value_counts())

Breed
Cold Water                27
Lizard Mix                11
Tarantula                  8
Lizard/Gecko               3
Frog                       1
Snake/Snake                1
Turtle/Redeared Slider     1
Name: count, dtype: int64


In [75]:
# Start with the most common, "Cold Water"

sus_manual.loc[sus_manual['Breed'] == 'Cold Water', :]

,Name,IntakeType,Breed,Color
11703,Plecostomus,Public Assist,Cold Water,Black/Gray
11704,Oscar,Public Assist,Cold Water,Gray/Orange
11705,Oscar,Public Assist,Cold Water,White/Orange
11708,Koi,Public Assist,Cold Water,Orange/White
11709,Plecostomus,Public Assist,Cold Water,Black/Gray
77896,nan,Public Assist,Cold Water,Black
77897,nan,Public Assist,Cold Water,Black
77898,nan,Public Assist,Cold Water,Black
77899,nan,Public Assist,Cold Water,Black
104527,nan,Public Assist,Cold Water,Black


From the names that are present, these look like pets, so I am going to omit that from our wildlife list.

In [76]:
# Next, "Lizard Mix"

sus_manual.loc[sus_manual['Breed'] == 'Lizard Mix', :]

,Name,IntakeType,Breed,Color
15294,nan,Stray,Lizard Mix,Tan
24402,nan,Public Assist,Lizard Mix,Tan
26912,nan,Public Assist,Lizard Mix,Black/Yellow
38017,nan,Stray,Lizard Mix,Green
38018,nan,Stray,Lizard Mix,Green
38019,nan,Stray,Lizard Mix,Green
41730,nan,Stray,Lizard Mix,Green
52667,nan,Stray,Lizard Mix,Brown
56219,nan,Stray,Lizard Mix,Tan
74843,nan,Stray,Lizard Mix,Tricolor


This is less straightforward, so I will have to use my judgement. "Stray" insinuates that these may be domestic animals, so I will omit this category from the wildlife list as well.

In [77]:
# I will do the rest all at once

sus_manual.loc[aacIn["Breed"].isin(['Tarantula', 'American', 'Lizard/Gecko',
                                    'Frog', 'Snake/Snake', 'Turtle/Redeared Slider']), :]

,Name,IntakeType,Breed,Color
32482,nan,Owner Surrender,Tarantula,Brown/Red
33074,nan,Public Assist,Frog,Brown
33075,nan,Public Assist,Tarantula,Brown
33076,nan,Public Assist,Tarantula,Brown
33077,nan,Public Assist,Tarantula,Brown
33078,nan,Public Assist,Tarantula,Brown
33079,nan,Public Assist,Tarantula,Brown
33080,nan,Public Assist,Snake/Snake,Tan/Brown
160647,nan,Stray,Turtle/Redeared Slider,Green/Black
165721,nan,Public Assist,Lizard/Gecko,Yellow/Brown


I'm going to have to intuit here a few things:
* Names mean they are not a wild animal
* "Owner Surrender" also means not wild
* "Public Assist" leans toward wildlife
* This leaves "Frog", "Snake/Snake", "Turtle/Redeared Slider", "Tarantula", and "Lizard/Gecko" as likely wildlife. 

In [78]:
# Add them to our wildlife list

wildlife.extend(["Frog", "Snake/Snake", "Turtle/Redeared Slider", "Lizard/Gecko", "Tarantula"])

# And now we see how many entries can be labeled "AnimalType" = "Wildlife"

aacIn.loc[aacIn["Breed"].isin(wildlife), ['Name', 'IntakeType', 'Breed', 'Color']]

,Name,IntakeType,Breed,Color
78,nan,Wildlife,Bat Mix,Brown
122,nan,Wildlife,Raccoon Mix,Brown
137,nan,Wildlife,Skunk,Black/White
142,nan,Wildlife,Raccoon Mix,Black/White
218,nan,Wildlife,Bat Mix,Brown/Brown
...,...,...,...,...
173180,nan,Wildlife,Raccoon,Gray/Black
173197,nan,Wildlife,Bat,Brown/Brown
173208,nan,Wildlife,Fox,Red
173209,nan,Wildlife,Raccoon,Black/Gray


In [79]:
# And change their AnimalType "Other" label to "Wildlife"

aacIn.loc[aacIn["Breed"].isin(wildlife) & (aacIn["AnimalType"] == "Other"), "AnimalType"] = "Wildlife"

# Check it worked
aacIn.loc[aacIn["Breed"].isin(wildlife), ['Name', 'IntakeType', 'Breed', 'AnimalType']]

,Name,IntakeType,Breed,AnimalType
78,nan,Wildlife,Bat Mix,Wildlife
122,nan,Wildlife,Raccoon Mix,Wildlife
137,nan,Wildlife,Skunk,Wildlife
142,nan,Wildlife,Raccoon Mix,Wildlife
218,nan,Wildlife,Bat Mix,Wildlife
...,...,...,...,...
173180,nan,Wildlife,Raccoon,Wildlife
173197,nan,Wildlife,Bat,Wildlife
173208,nan,Wildlife,Fox,Wildlife
173209,nan,Wildlife,Raccoon,Wildlife


In [80]:
# There seemed to be a lot of rabbit breeds. Let's see if it is worth changing their category.

rabbits = [
    'Rabbit Sh Mix', 'Rabbit Sh', 'Californian Mix', 'Californian',
    'Rabbit Sh/Lop-Mini', 'Rex Mix', 'Jersey Wooly Mix', 'Cottontail Mix',
    'Lop-English Mix', 'American Mix', 'Lop-Holland Mix', 'Lionhead',
    'Rabbit Lh Mix', 'Dutch/Rabbit Sh', 'Hotot Mix', 'Rabbit Lh',
    'American Sable Mix', 'Angora-French Mix', 'Dutch', 'Netherlnd Dwarf Mix',
    'Checkered Giant Mix', 'Lop-Mini', 'Havana Mix', 'Chinchilla-Amer Mix',
    'Chinchilla-Stnd Mix', 'Polish Mix', 'English Spot', 'Harlequin Mix',
    'Netherlnd Dwarf', 'Chinchilla Mix', 'Rex-Mini', 'Rex',
    'Lop-Amer Fuzzy Mix', 'Rex-Mini Mix', 'Dutch/Angora-Satin',
    'Flemish Giant', 'Belgian Hare', 'Britannia Petit Mix',
    'Himalayan Mix', 'Flemish Giant Mix', 'Lop-Amer Fuzzy', 'Rhinelander Mix',
    'Belgian Hare Mix', 'Lop-Mini/Lop-Holland', 'Beveren Mix',
    'Angora-Satin Mix', 'Rex/Lop-English', 'Havana', 'Polish',
    'Chinchilla-Amer', 'Dwarf Hotot', 'Angora-Satin', 'Cottontail',
    'Lop-English', 'Lop-Mini/Hotot', 'Chinchilla-Stnd', 'Jersey Wooly',
    'Angora-English', 'New Zealand Wht/Lop-Holland', 'Rabbit Sh/Dwarf Hotot',
    'Himalayan', 'Florida White', 'Lop-French', 'New Zealand Wht', 'American',
    'Cinnamon', 'Cinnamon Mix'
]

aacIn.loc[aacIn["Breed"].isin(rabbits), ['Name', 'IntakeType', 'Breed', 'AnimalType']]

,Name,IntakeType,Breed,AnimalType
112,Birch,Stray,Rabbit Sh Mix,Other
237,Dante,Owner Surrender,Rabbit Sh Mix,Other
350,nan,Stray,Himalayan Mix,Cat
518,nan,Stray,Californian Mix,Other
519,nan,Stray,Californian Mix,Other
...,...,...,...,...
172248,nan,Stray,Rabbit Sh,Other
172506,Lottie,Owner Surrender,Rabbit Sh,Other
172507,Lola,Owner Surrender,Rabbit Sh,Other
172537,Napoleon,Stray,Rabbit Sh,Other


Out of 170,000 entries, I don't think 1000 is hugely significant; however, the number of breeds which are rabbits is relatively long (compared to the other creatures in "Other"). I believe I will give them their own category anyway.

In [81]:
# Let's find some rabbits

other_breeds = aacIn.loc[aacIn['AnimalType'] == 'Other', 'Breed'].unique()
pprint(other_breeds)

array(['Rabbit Sh Mix', 'Ferret', 'Californian Mix', 'Rabbit Sh',
       'Guinea Pig Mix', 'Rabbit Sh/Lop-Mini', 'Rex Mix', 'Guinea Pig',
       'Californian', 'Hamster Mix', 'English Spot Mix', 'Lionhead Mix',
       'Ferret Mix', 'Silver Mix', 'Sugar Glider', 'Dutch Mix',
       'Jersey Wooly Mix', 'Lop-English Mix', 'American Mix',
       'Lop-Holland Mix', 'Lionhead', 'Rabbit Lh Mix', 'Dutch/Rabbit Sh',
       'American Sable', 'New Zealand Wht Mix', 'Lop-Mini Mix',
       'Hotot Mix', 'Hamster', 'Cinnamon', 'Rabbit Lh',
       'American Sable Mix', 'Gerbil Mix', 'Angora-French Mix', 'Dutch',
       'Netherlnd Dwarf Mix', 'Rex-Mini', 'Lop-Holland',
       'Checkered Giant Mix', 'Lop-Mini', 'Havana Mix', 'Hedgehog',
       'Chinchilla-Amer Mix', 'Chinchilla-Stnd Mix', 'Polish Mix',
       'English Spot', 'Harlequin Mix', 'Netherlnd Dwarf',
       'Chinchilla Mix', 'Rex-Mini/Lop-English', 'Angora-English Mix',
       'Cinnamon Mix', 'Rex', 'Lop-Amer Fuzzy Mix', 'Rex-Mini Mix',
      

Manually picking out the rabbit breeds, I see there is a lot of shorthand, some misspellings, and a lot of inconsistency in how they are written. The breeds will require some significant cleaning and processing soon, I can already tell.

In [82]:
# For now, we compile what I can identify as rabbit breeds

bun_breeds = ['Rabbit Sh Mix', 'Californian Mix', 'Rabbit Sh', 'Rabbit Sh/Lop-Mini', 'Rex Mix', 
        'Californian', 'English Spot Mix', 'Lionhead Mix', 'Silver Mix', 'Dutch Mix', 
        'Jersey Wooly Mix', 'Lop-English Mix', 'American Mix', 'Lop-Holland Mix', 'Lionhead', 
        'Rabbit Lh Mix', 'Dutch/Rabbit Sh', 'American Sable', 'New Zealand Wht Mix', 'Lop-Mini Mix', 
        'Hotot Mix', 'Rabbit Lh', 'American Sable Mix', 'Angora-French Mix', 'Dutch', 'Netherlnd Dwarf Mix', 
        'Rex-Mini', 'Lop-Holland', 'Checkered Giant Mix', 'Lop-Mini', 'Havana Mix', 'Polish Mix', 
        'English Spot', 'Harlequin Mix', 'Netherlnd Dwarf', 'Rex-Mini/Lop-English', 'Angora-English Mix', 
        'Rex', 'Lop-Amer Fuzzy Mix', 'Rex-Mini Mix', 'Dutch/Angora-Satin', 'Flemish Giant', 'Belgian Hare', 
        'Britannia Petit Mix', 'Himalayan Mix', 'Lop-Amer Fuzzy', 'Rhinelander Mix', 'Belgian Hare Mix', 
        'Flemish Giant Mix', 'Lop-Mini/Lop-Holland', 'Beveren Mix', 'Angora-Satin Mix', 'Rex/Lop-English', 
        'Havana', 'Polish', 'New Zealand Wht', 'Hotot', 'Dwarf Hotot', 'Angora-Satin', 'Lop-English', 
        'Lop-Mini/Hotot', 'Jersey Wooly', 'Angora-English', 'New Zealand Wht/Lop-Holland', 
        'Rabbit Sh/Dwarf Hotot', 'Himalayan', 'Florida White', 'Lop-French', 'Rabbit Sh/Californian', 
       'American', 'American Mix', 'Cinnamon', 'Cinnamon Mix']

In [83]:
# Give them their new category!

aacIn.loc[aacIn["Breed"].isin(bun_breeds) & (aacIn["AnimalType"] == "Other"), "AnimalType"] = "Rabbit"

# Check it worked

aacIn.loc[aacIn["Breed"].isin(bun_breeds), ['Name', 'IntakeType', 'Breed', 'AnimalType']]

,Name,IntakeType,Breed,AnimalType
112,Birch,Stray,Rabbit Sh Mix,Rabbit
237,Dante,Owner Surrender,Rabbit Sh Mix,Rabbit
350,nan,Stray,Himalayan Mix,Cat
518,nan,Stray,Californian Mix,Rabbit
519,nan,Stray,Californian Mix,Rabbit
...,...,...,...,...
172506,Lottie,Owner Surrender,Rabbit Sh,Rabbit
172507,Lola,Owner Surrender,Rabbit Sh,Rabbit
172537,Napoleon,Stray,Rabbit Sh,Rabbit
172794,nan,Stray,Rabbit Sh/Californian,Rabbit


In [84]:
# Let's double-check our work here:

print(aacIn['AnimalType'].value_counts())

AnimalType
Dog          94406
Cat          69009
Wildlife      6948
Rabbit        1071
Other          914
Bird           876
Livestock       33
Name: count, dtype: int64


Out of originally 9000 "Other" entries, 7000 were wildlife, so I believe separating them was a good decision. The rabbits are more debatable, but it makes the list of "Other" breeds much more distinct and consistent.

#### Outcomes

In [85]:
# Are these the same as the intake categories?

print(aacOut['AnimalType'].value_counts())

AnimalType
Dog          94188
Cat          69047
Other         8909
Bird           874
Livestock       34
Name: count, dtype: int64


In [86]:
# They are. So, let's do the same things we did earlier
# First for wildlife...

aacOut.loc[aacOut["Breed"].isin(wildlife) & (aacOut["AnimalType"] == "Other"), "AnimalType"] = "Wildlife"

# Check it worked
aacOut.loc[aacOut["Breed"].isin(wildlife), ['Name', 'OutcomeType', 'Breed', 'AnimalType']]

,Name,OutcomeType,Breed,AnimalType
3,nan,Euthanasia,Raccoon,Wildlife
24,nan,Euthanasia,Raccoon,Wildlife
29,nan,Euthanasia,Opossum,Wildlife
41,nan,Euthanasia,Bat Mix,Wildlife
47,nan,Euthanasia,Bat Mix,Wildlife
...,...,...,...,...
172831,nan,Euthanasia,Bat Mix,Wildlife
172885,nan,Euthanasia,Raccoon,Wildlife
173064,nan,Euthanasia,Raccoon,Wildlife
173065,nan,Euthanasia,Bat,Wildlife


In [87]:
# Now let's help out the rabbits...

aacOut.loc[aacOut["Breed"].isin(bun_breeds) & (aacOut["AnimalType"] == "Other"), "AnimalType"] = "Rabbit"

# Check it worked
aacOut.loc[aacOut["Breed"].isin(bun_breeds), ['Name', 'OutcomeType', 'Breed', 'AnimalType']]

,Name,OutcomeType,Breed,AnimalType
60,Moo,Adoption,Polish,Rabbit
243,nan,Transfer,Rabbit Sh Mix,Rabbit
540,nan,Transfer,Rabbit Sh Mix,Rabbit
698,Peter,Adoption,Rabbit Sh,Rabbit
1395,nan,Euthanasia,Rabbit Sh,Rabbit
...,...,...,...,...
172475,Sweet Potato,Adoption,Rabbit Sh,Rabbit
172487,nan,Transfer,Rabbit Sh,Rabbit
172805,Floppy,Transfer,Rabbit Sh Mix,Rabbit
172975,nan,Transfer,Rabbit Sh/Californian,Rabbit


In [88]:
# Let's see what we're looking at now

print(aacIn['AnimalType'].value_counts())

AnimalType
Dog          94406
Cat          69009
Wildlife      6948
Rabbit        1071
Other          914
Bird           876
Livestock       33
Name: count, dtype: int64


In [89]:
# Since I didn't manually check the "Other" category for this dataset, I want to now,
    # in case any breed values are different or new

aacOut.loc[(aacOut['AnimalType'] == 'Other'), 'Breed'].unique()

array(['Guinea Pig Mix', 'Guinea Pig', 'Ferret Mix', 'Chinchilla',
       'Hamster', 'Ferret', 'Chinchilla Mix', 'Hamster Mix', 'Hedgehog',
       'Snake/Python', 'Chinchilla-Stnd', 'Gerbil', 'Tropical',
       'Gerbil Mix', 'Lizard/Bearded Dragon', 'Tropical Mix',
       'Chinchilla-Amer', 'Hermit Crab Mix', 'Chinchilla-Stnd Mix',
       'Sugar Glider', 'Chinchilla-Amer Mix'], dtype=object)

Looks like there are no missed wildlife or rabbits; onward!

### Sex

#### IntakeSex

In [90]:
print(aacIn['IntakeSex'].value_counts())

IntakeSex
Intact Male      58813
Intact Female    56512
Neutered Male    24190
Spayed Female    20280
Unknown          13460
Name: count, dtype: int64


These categories look fine to me.

#### OutcomeSex

In [91]:
print(aacOut['OutcomeSex'].value_counts())

OutcomeSex
Neutered Male    60681
Spayed Female    55062
Intact Male      22167
Intact Female    21731
Unknown          13409
Name: count, dtype: int64


The value counts have more or less reversed compared to the IntakeSex category, which is good! That just means many animals were fixed while in shelter. I suspect most of these are cats and dogs.

### Age

#### IntakeAge

In [92]:
print(aacIn['IntakeAge'].value_counts())

IntakeAge
1 year       28202
2 years      27951
1 month      18148
2 months     10409
3 years       9579
4 weeks       6295
4 years       5826
5 years       5177
3 weeks       5101
3 months      4900
4 months      4773
5 months      4368
2 weeks       3650
6 years       3421
6 months      3325
7 years       3001
8 years       2829
7 months      2585
9 months      2487
10 years      2288
8 months      2025
9 years       1670
1 week        1469
1 weeks       1413
10 months     1383
0 years       1247
11 months     1137
12 years      1116
1 day         1009
11 years       923
3 days         920
2 days         768
13 years       715
6 days         532
14 years       523
4 days         497
15 years       451
5 weeks        444
5 days         263
16 years       192
17 years       104
18 years        62
19 years        29
20 years        26
22 years         6
-1 years         6
25 years         2
-3 years         2
-2 years         1
24 years         1
-4 years         1
21 years         1
23

The negative ages seem probably incorrect, and I think age bins could be helpful for graphing and modeling. 

In [93]:
# Remove leading/trailing white spaces and those negatives

aacIn["IntakeAge"] = aacIn["IntakeAge"].str.replace(r"^-", "", regex=True)

Making that binned column will take a few steps here.

In [94]:
# Convert age string to years; making this a function for use on the Outcomes dataset as well

def age_to_years(age_str):
    """
    Converts age strings to numerical years as floats.
    
    Args:
        age_str (str): String representing an age ("3 weeks", "4 months", "2 years"...).

    Returns:
        float: Float value representing that age in years.
    """
    match = re.match(r"(-?\d+)\s*(\w+)", str(age_str).lower())
    if not match:
        return np.nan
    value, unit = int(match.group(1)), match.group(2)

    if "year" in unit:
        return abs(value)
    elif "month" in unit:
        return abs(value) / 12
    elif "week" in unit:
        return abs(value) / 52
    elif "day" in unit:
        return abs(value) / 365
    else:
        return np.nan

In [95]:
# Convert intake age to numeric years to calculate bins

aacIn['AgeYears'] = aacIn['IntakeAge'].apply(age_to_years)

In [96]:
# Bin into age categories, again creating a function

def age_bins(age):
    """
    Converts float- or integer-form ages into age categories.
    
    Args:
        age (float/int): Numerical representation of age in years.

    Returns:
        str: Corresponding age category ("Unknown", "Infant", "Young", "Adult", "Senior").
    """
    if pd.isna(age):
        return "Unknown"
    elif age < 1:
        return "Infant"
    elif 1 <= age <= 2:
        return "Young"
    elif 3 <= age <= 7:
        return "Adult"
    else:
        return "Senior"

In [97]:
# Execute function to create a column of age categories

aacIn["AgeCategory"] = aacIn["AgeYears"].apply(age_bins)

In [98]:
# Round the AgeYears column so that it can be an integer for simplicity

aacIn['AgeYears'] = aacIn['AgeYears'].round().astype('Int64')

In [99]:
# Check that this worked

print(aacIn['AgeCategory'].value_counts())

AgeCategory
Infant     79148
Young      56160
Adult      27007
Senior     10941
Unknown        1
Name: count, dtype: int64


In [100]:
# For now, I'll keep that intermediary column--it likely will be more helpful than the original age category 
# (even if it loses some information through rounding)

print(aacIn['AgeYears'].value_counts())

AgeYears
0     69531
1     37825
2     27952
3      9581
4      5827
5      5177
6      3421
7      3001
8      2829
10     2288
9      1670
12     1116
11      923
13      715
14      523
15      451
16      192
17      104
18       62
19       29
20       26
22        6
25        2
24        1
21        1
23        1
30        1
28        1
Name: count, dtype: Int64


#### OutcomeAge

In [101]:
# Preview what this category looks like

print(aacOut['OutcomeAge'].value_counts())

OutcomeAge
1 year       28667
2 years      27793
2 months     23182
3 years       9758
3 months      9174
1 month       7378
4 months      6037
4 years       5797
5 years       5249
5 months      4429
6 months      4184
6 years       3416
7 years       3009
8 years       2950
3 weeks       2885
2 weeks       2877
8 months      2706
4 weeks       2407
10 months     2392
10 years      2382
7 months      2269
9 months      1763
9 years       1656
1 weeks       1345
12 years      1161
11 months     1099
1 week         961
11 years       922
13 years       743
3 days         567
14 years       535
2 days         526
1 day          506
15 years       462
4 days         357
6 days         341
0 years        272
5 days         235
5 weeks        202
16 years       200
17 years       107
18 years        64
19 years        29
20 years        26
22 years         6
-1 years         6
-3 years         2
25 years         2
-2 years         1
28 years         1
23 years         1
30 years         1
2

In [102]:
# Remove those negatives and other irregularities

aacOut["OutcomeAge"] = aacOut["OutcomeAge"].str.replace(r"^-", "", regex=True)

In [103]:
# Convert age to numeric years for calculating bins and other evaluations

aacOut['AgeYears'] = aacOut['OutcomeAge'].apply(age_to_years)

In [104]:
# Create a column of age categories

aacOut["AgeCategory"] = aacOut["AgeYears"].apply(age_bins)

In [105]:
# Round AgeYears to integer

aacOut['AgeYears'] = aacOut['AgeYears'].round().astype('Int64')

In [106]:
# Check the new columns

print(aacOut['AgeCategory'].value_counts())

AgeCategory
Infant     78094
Young      56467
Adult      27232
Senior     11250
Unknown        9
Name: count, dtype: int64


In [107]:
# Also AgeYears

print(aacOut['AgeYears'].value_counts())

AgeYears
0     67865
1     38902
2     27794
3      9760
4      5798
5      5249
6      3416
7      3009
8      2950
10     2382
9      1656
12     1161
11      922
13      743
14      535
15      462
16      200
17      107
18       64
19       29
20       26
22        6
25        2
28        1
23        1
30        1
24        1
21        1
Name: count, dtype: Int64


It is slightly odd that there are more Unknowns in the Outcome dataset than Intake, but not enough to be worrying Intake ages are likely a guess, anyway.

### Breed

#### Intakes

##### Preliminary Cleaning

In [108]:
# What does it look like right now?

print(aacIn['Breed'].value_counts())

Breed
Domestic Shorthair Mix                    33653
Domestic Shorthair                        23903
Pit Bull Mix                              10084
Labrador Retriever Mix                     8802
Chihuahua Shorthair Mix                    6880
                                          ...  
Queensland Heeler/Border Collie               1
Queensland Heeler/Cardigan Welsh Corgi        1
Manchester Terrier/Basenji                    1
Border Collie/Miniature Pinscher              1
Dachshund Longhair/Border Collie              1
Name: count, Length: 3003, dtype: int64


This is a lot of unique breeds. 3003 is too much for using in much of anything. My thought is to make a category, "Mix", which specifies whether the animal is a mixed-breed, and another column that specifies a primary breed. I would like to keep secondary breeds as well, but suspect there will be too many nulls to justify keeping.

In [109]:
# Strip capitals first, to make sure there are no case-related misspellings

aacIn['Breed'] = aacIn['Breed'].str.lower().str.strip()

print(aacIn['Breed'].value_counts())
print('\n')

# And return those capitals

aacIn['Breed'] = aacIn['Breed'].str.title()

print(aacIn['Breed'].value_counts())

Breed
domestic shorthair mix                    33653
domestic shorthair                        23903
pit bull mix                              10084
labrador retriever mix                     8802
chihuahua shorthair mix                    6880
                                          ...  
queensland heeler/border collie               1
queensland heeler/cardigan welsh corgi        1
manchester terrier/basenji                    1
border collie/miniature pinscher              1
dachshund longhair/border collie              1
Name: count, Length: 3003, dtype: int64


Breed
Domestic Shorthair Mix                    33653
Domestic Shorthair                        23903
Pit Bull Mix                              10084
Labrador Retriever Mix                     8802
Chihuahua Shorthair Mix                    6880
                                          ...  
Queensland Heeler/Border Collie               1
Queensland Heeler/Cardigan Welsh Corgi        1
Manchester Terrier/Basenji        

In [110]:
# Now to process this chaos

def parse_breeds(breed):
    """
    Cleans breed entries to separate, easier-to-handle columns.

    Removes "Mix" and creates Boolean column representing whether or not the animal is a mixed-breed; 
    splits entries along "/", and separates breeds into Primary and Secondary columns.
        
    Args:
        breed (str): String representing a breed or combination of breeds. Can include "Mix" or a combination of breeds, separated by "/".

    Returns:
        Series: Three (3) columns representing whether the animal is mixed-breed, as well as its primary and secondary breeds
        (by default: "is_mix", "primary", "secondary").
    """
    # Create "is_mix" Boolean column
    
    is_mix = 'Mix' in breed or '/' in breed

    # Remove "Mix"
    
    breed_clean = breed.replace(' Mix', '').strip()
    
    # Handle slashes and separate primary from secondary breeds
    
    if '/' in breed_clean:
        parts = breed_clean.split('/')
        primary = parts[0].strip()
        secondary = parts[1].strip() if len(parts) > 1 else None
    else:
        primary = breed_clean
        secondary = None

    # Return pandas Series with 3 columns

    return pd.Series([is_mix, primary, secondary])

In [111]:
# Apply to dataframe

aacIn[['IsMix', 'PrimaryBreed', 'SecondaryBreed']] = aacIn['Breed'].apply(parse_breeds)

In [112]:
aacIn[['AnimalType', 'Breed', 'IsMix', 'PrimaryBreed', 'SecondaryBreed']].head(10)

,AnimalType,Breed,IsMix,PrimaryBreed,SecondaryBreed
0,Dog,Border Terrier/Border Collie,True,Border Terrier,Border Collie
1,Cat,Domestic Shorthair Mix,True,Domestic Shorthair,None
2,Cat,Domestic Shorthair Mix,True,Domestic Shorthair,None
3,Cat,Domestic Shorthair Mix,True,Domestic Shorthair,None
4,Dog,Pit Bull Mix,True,Pit Bull,None
5,Cat,American Shorthair Mix,True,American Shorthair,None
6,Dog,Border Collie Mix,True,Border Collie,None
7,Cat,Domestic Shorthair Mix,True,Domestic Shorthair,None
8,Dog,Podengo Pequeno Mix,True,Podengo Pequeno,None
9,Dog,Chihuahua Shorthair Mix,True,Chihuahua Shorthair,None


Seems to have worked. Let's look at those new breed categories now.

In [113]:
print(aacIn['PrimaryBreed'].value_counts())
print("Primary breed null values: " + str(aacIn['PrimaryBreed'].isnull().sum()))
print('\n')
print(aacIn['SecondaryBreed'].value_counts())
print('Secondary breed null values: ' + str(aacIn['SecondaryBreed'].isnull().sum()))

PrimaryBreed
Domestic Shorthair     57573
Pit Bull               14494
Labrador Retriever     13530
Chihuahua Shorthair    10068
German Shepherd         6997
                       ...  
Rhinelander                1
Other Bird                 1
Hermit Crab                1
Beveren                    1
Pelican                    1
Name: count, Length: 385, dtype: int64
Primary breed null values: 0


SecondaryBreed
Labrador Retriever     2297
Pit Bull               1188
Chihuahua Shorthair    1080
German Shepherd        1041
Dachshund               725
                       ... 
Bengal                    1
Dandie Dinmont            1
Yorkshire                 1
British Shorthair         1
Californian               1
Name: count, Length: 211, dtype: int64
Secondary breed null values: 157661


385 primary breeds and 211 secondary breeds is far better than the 3000 we started with. However, I suspect there are still far too many categories with very few entries. Also looking at the number of null entries in "SecondaryBreed", I suspect there are just too many nulls to work with. This is ultimately an adoption-oriented project, and breed plays a huge part in that; however, there just is not enough information in that column to warrant keeping. Further, there are SecondaryBreeds for creatures such as wildlife which don't make sense and would also have to be removed, creating even more nulls.

In [114]:
# Drop that SecondaryBreed column

aacIn.drop(columns = 'SecondaryBreed', inplace = True)

# Check the dataset, as always

aacIn.head()

,AnimalID,Name,IntakeDT,IntakeType,IntakeCondition,AnimalType,IntakeSex,IntakeAge,Breed,Color,AgeYears,AgeCategory,IsMix,PrimaryBreed
0,A521520,Nina,2013-10-01 07:51:00,Stray,Normal,Dog,Spayed Female,7 years,Border Terrier/Border Collie,White/Tan,7,Adult,True,Border Terrier
1,A664235,nan,2013-10-01 08:33:00,Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White,0,Infant,True,Domestic Shorthair
2,A664236,nan,2013-10-01 08:33:00,Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White,0,Infant,True,Domestic Shorthair
3,A664237,nan,2013-10-01 08:33:00,Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White,0,Infant,True,Domestic Shorthair
4,A664233,Stevie,2013-10-01 08:53:00,Stray,Injured,Dog,Intact Female,3 years,Pit Bull Mix,Blue/White,3,Adult,True,Pit Bull


Now for the hard part: finding misspellings, shorthand notations, and other general inconsistencies in these breeds. To do so, I am going to first divide the breeds into animal types, and work through each one at a time.

##### Manually Checking and Replacing Breeds

In [115]:
# Starting with the dogs, of course

print(aacIn.loc[aacIn['AnimalType'] == 'Dog', 'PrimaryBreed'].nunique())

211


In [116]:
# I will go through this list manually, and then use ChatGPT for a sanity check

pprint(aacIn.loc[aacIn['AnimalType'] == 'Dog', 'PrimaryBreed'].unique().tolist())

['Border Terrier',
 'Pit Bull',
 'Border Collie',
 'Podengo Pequeno',
 'Chihuahua Shorthair',
 'Weimaraner',
 'Anatol Shepherd',
 'Labrador Retriever',
 'Great Pyrenees',
 'Cairn Terrier',
 'Dachshund',
 'German Shepherd',
 'Black',
 'Siberian Husky',
 'Miniature Poodle',
 'Yorkshire Terrier',
 'Norfolk Terrier',
 'Australian Shepherd',
 'Australian Cattle Dog',
 'Belgian Malinois',
 'Jack Russell Terrier',
 'Doberman Pinsch',
 'Japanese Chin',
 'Staffordshire',
 'Welsh Terrier',
 'Shetland Sheepdog',
 'Vizsla',
 'Rottweiler',
 'American Staffordshire Terrier',
 'American Bulldog',
 'Chinese Sharpei',
 'Great Dane',
 'Harrier',
 'Flat Coat Retriever',
 'Dachshund Wirehair',
 'Leonberger',
 'Dachshund Longhair',
 'Chesa Bay Retr',
 'Mastiff',
 'Chihuahua Longhair',
 'Rat Terrier',
 'Greyhound',
 'Boxer',
 'Bulldog',
 'Miniature Pinscher',
 'Carolina Dog',
 'Toy Poodle',
 'Pointer',
 'Queensland Heeler',
 'American Pit Bull Terrier',
 'Miniature Schnauzer',
 'Beagle',
 'Kuvasz',
 'Shih T

There are some very unusual breeds in here. However, all or almost all of these are probably guesses from shelter staff. Eventually, I will cull the least common breeds from the dataset; I will leave that until after the join, though.

Using the list of mistypes and standardization errors that I was able to catch manually (with ChatGPT to double-check my work), I will make a dictionary with the breed names that need to be replaced, along with their replacements.

In [117]:
# Dictionary with breed names that need to be replaced

dog_replacements = {
    'Black' : 'Other', 
    'Doberman Pinsch' : 'Doberman Pinscher', 
    'Chinese Sharpei' : 'Chinese Shar-Pei', 
    'Dachshund Wirehair' : 'Dachshund', 
    'Chihuahua Shorthair' : 'Chihuahua', 
    'Flat Coat Retriever' : 'Flat-Coated Retriever', 
    'Dachshund Longhair' : 'Dachshund', 
    'Chihuahua Longhair' : 'Chihuahua', 
    'Pointer' : 'German Shorthaired Pointer', 
    'Catahoula' : 'Catahoula Leopard Dog',
    'Parson Russell Terrier' : 'Jack Russell Terrier', 
    'Port Water Dog' : 'Portuguese Water Dog', 
    'Miniature Bull Terrier' : 'Bull Terrier', 
    'Bullmastiff' : 'Mastiff', 
    'Eng Toy Spaniel' : 'English Toy Spaniel', 
    'Wire Hair Fox Terrier' : 'Wire Fox Terrier', 
    'West Highland' : 'West Highland White Terrier', 
    'Bruss Griffon' : 'Brussels Griffon', 
    'Petit Basset Griffon Vendéen' : 'Petit Basset Griffon Vendeen', 
    'German Shorthair Pointer' : 'German Shorthaired Pointer', 
    'Cavalier Span' : 'Cavalier King Charles Spaniel', 
    'Bluetick Hound' : 'Bluetick Coonhound', 
    'Boykin Span' : 'Boykin Spaniel', 
    'Chinese Crested' : 'Chinese Crested Dog', 
    'Glen Of Imaal' : 'Glen Of Imaal Terrier', 
    'Patterdale Terr' : 'Patterdale Terrier', 
    'Dandie Dinmont' : 'Dandie Dinmont Terrier', 
    'Schnauzer Giant' : 'Giant Schnauzer', 
    'Sealyham Terr' : 'Sealyham Terrier', 
    'Bedlington Terr' : 'Bedlington Terrier', 
    'Sussex Span' : 'Sussex Spaniel', 
    'Wirehaired Vizsla' : 'Vizsla', 
    'Curly Coat Retriever' : 'Curly-Coated Retriever',
    'Pit Bull': 'American Pit Bull Terrier',
    'Anatol Shepherd': 'Anatolian Shepherd',
    'Staffordshire': 'Staffordshire Bull Terrier',
    'Rhod Ridgeback': 'Rhodesian Ridgeback',
    'Chesa Bay Retr': 'Chesapeake Bay Retriever',
    'Queensland Heeler': 'Australian Cattle Dog',
    'Collie Rough': 'Rough Collie',
    'Collie Smooth': 'Smooth Collie',
    'St. Bernard Rough Coat': 'Saint Bernard',
    'St. Bernard Smooth Coat': 'Saint Bernard',
    'Pbgv': 'Petit Basset Griffon Vendeen',
    'Bull Terrier Miniature': 'Bull Terrier',
    'Mexican Hairless': 'Xoloitzcuintli',
    'Bouv Flandres': 'Bouvier des Flandres',
    'Bernese Hound': 'Bernese Mountain Dog',
    'Silkie': 'Silky Terrier',
    'Dachshund Stan': 'Dachshund',
    'Unknown': 'Other',
    'Yorkshire' : 'Yorkshire Terrier',
    'English Cocker Spaniel' : 'Cocker Spaniel',
    'Papillion' : 'Papillon',
    'Tan Hound': 'Black Mouth Cur'
}

# That's a lot of changes

In [118]:
# Make a function to replace breeds in the "PrimaryBreed" column, since I will need this for every animal type in both datasets.

def replace_breeds(df, animal_type, replacement_dict):
    """
    Taking a dictionary of current values and desired replacements, 
    replaces designated values in the "PrimaryBreed" column of the specified Dataframe.

    Prints feedback for debugging and monitoring.

    Args:
        df (pd.DataFrame): Dataframe within which the column to edit lies.
        animal_type (str): String representing an animal type found in the "AnimalType" column (a column specific to this project).
        replacement_dict (dict): Dictionary in the form of {"Current Value" : "Replacement Value"}.

    Returns:
        pd.DataFrame: the modified Dataframe (inplace replacements).
    """
    # Initialize total replacements counter
    
    total_replacements = 0  # For monitoring the changes the function made
    
    # Filter for rows where AnimalType matches the provided animal_type
    
    filtered_df = df.loc[df['AnimalType'] == animal_type]  # This way, each animal type can be evaluated individually
    
    # Apply the replacement for each breed in the specified columns
    
    for old_value, new_value in replacement_dict.items():
            
        # Count the occurrences before replacement
            
        old_count = df.loc[(df['AnimalType'] == animal_type) & (df['PrimaryBreed'] == old_value), 'PrimaryBreed'].shape[0]
            
        # Debugging: Check if old_value exists
            
        print(f"Checking for '{old_value}' in 'PrimaryBreed'. Found: {old_count} times.")
        if old_count == 0:
            print()
            
        # Only perform replacement if old_value exists in the dataframe for that animal
            
        if old_count > 0:
            df.loc[(df['AnimalType'] == animal_type) & (df['PrimaryBreed'] == old_value), 'PrimaryBreed'] = new_value  # Actual replacement
            total_replacements += old_count  # Keeping track of how many replacements have been made
            print(f"Replaced {old_count} occurrences of '{old_value}' with '{new_value}'.")  # Debugging and monitoring feedback
            print()

    # Get unique breeds in PrimaryBreed after replacement
        
    unique_breeds = df.loc[df['AnimalType'] == animal_type, 'PrimaryBreed'].nunique()
    print(f"After replacements, the unique breeds in 'PrimaryBreed' for {animal_type} are now: {unique_breeds}")
    
    # Print final feedback
    
    print(f"\nTotal replacements made: {total_replacements}")
    
    # Return the modified dataframe
    
    return df

In [119]:
# Function made, let's replace some breeds

aacIn = replace_breeds(aacIn, "Dog", dog_replacements)

Checking for 'Black' in 'PrimaryBreed'. Found: 233 times.
Replaced 233 occurrences of 'Black' with 'Other'.

Checking for 'Doberman Pinsch' in 'PrimaryBreed'. Found: 543 times.
Replaced 543 occurrences of 'Doberman Pinsch' with 'Doberman Pinscher'.

Checking for 'Chinese Sharpei' in 'PrimaryBreed'. Found: 315 times.
Replaced 315 occurrences of 'Chinese Sharpei' with 'Chinese Shar-Pei'.

Checking for 'Dachshund Wirehair' in 'PrimaryBreed'. Found: 223 times.
Replaced 223 occurrences of 'Dachshund Wirehair' with 'Dachshund'.

Checking for 'Chihuahua Shorthair' in 'PrimaryBreed'. Found: 10068 times.
Replaced 10068 occurrences of 'Chihuahua Shorthair' with 'Chihuahua'.

Checking for 'Flat Coat Retriever' in 'PrimaryBreed'. Found: 159 times.
Replaced 159 occurrences of 'Flat Coat Retriever' with 'Flat-Coated Retriever'.

Checking for 'Dachshund Longhair' in 'PrimaryBreed'. Found: 258 times.
Replaced 258 occurrences of 'Dachshund Longhair' with 'Dachshund'.

Checking for 'Chihuahua Longhair' 

In [120]:
# Moving on to the cats

pprint(aacIn.loc[aacIn['AnimalType'] == 'Cat', 'PrimaryBreed'].unique().tolist())

['Domestic Shorthair',
 'American Shorthair',
 'Russian Blue',
 'Domestic Longhair',
 'Domestic Medium Hair',
 'Manx',
 'Himalayan',
 'Snowshoe',
 'Siamese',
 'Abyssinian',
 'Maine Coon',
 'Bengal',
 'Balinese',
 'Persian',
 'Tonkinese',
 'British Shorthair',
 'Exotic Shorthair',
 'Bombay',
 'Ragdoll',
 'Burmese',
 'Angora',
 'Japanese Bobtail',
 'Norwegian Forest Cat',
 'Sphynx',
 'Cymric',
 'Pixiebob Shorthair',
 'Munchkin Shorthair',
 'Devon Rex',
 'Chartreux',
 'Turkish Van',
 'American Wirehair',
 'Oriental Sh',
 'Turkish Angora',
 'Javanese',
 'Havana Brown',
 'Cornish Rex',
 'Scottish Fold',
 'Munchkin Longhair',
 'Ocicat',
 'American Curl Shorthair',
 'Birman',
 'Colorpoint',
 'Savannah',
 'Rex']


In [121]:
# Make our corrections dictionary

cat_replacements = {
    'Oriental Sh' : 'Oriental Shorthair',
    'Angora' : 'Turkish Angora', 
    'Colorpoint' : 'Colorpoint Shorthair', 
    'Rex' : 'Cornish Rex',
}

In [122]:
# Replacing breeds again

aacIn = replace_breeds(aacIn, "Cat", cat_replacements)

Checking for 'Oriental Sh' in 'PrimaryBreed'. Found: 5 times.
Replaced 5 occurrences of 'Oriental Sh' with 'Oriental Shorthair'.

Checking for 'Angora' in 'PrimaryBreed'. Found: 13 times.
Replaced 13 occurrences of 'Angora' with 'Turkish Angora'.

Checking for 'Colorpoint' in 'PrimaryBreed'. Found: 3 times.
Replaced 3 occurrences of 'Colorpoint' with 'Colorpoint Shorthair'.

Checking for 'Rex' in 'PrimaryBreed'. Found: 1 times.
Replaced 1 occurrences of 'Rex' with 'Cornish Rex'.

After replacements, the unique breeds in 'PrimaryBreed' for Cat are now: 42

Total replacements made: 22


In [123]:
# Evaluating birds

print(aacIn.loc[aacIn['AnimalType'] == 'Bird', 'PrimaryBreed'].unique().tolist())

['Pigeon', 'Parakeet', 'Rhode Island', 'Chicken', 'Duck', 'Lark', 'Cockatiel', 'Bantam', 'Lovebird', 'Parrot', 'Sparrow', 'Catbird', 'Grackle', 'Nuthatch', 'Quaker', 'Dove', 'Silkie', 'Finch', 'Turkey', 'Guinea', 'Conure', 'Bluebird', 'Bunting', 'Mockingbird', 'Owl', 'Goose', 'Budgerigar', 'Hawk', 'Wren', 'Peafowl', 'Vulture', 'Heron', 'Cockatoo', 'African', 'Barred Rock', 'Crow', 'Muscovy', 'Song Bird', 'Pheasant', 'Leghorn', 'Chickadee', 'Whimbrel', 'Other Bird', 'Black', 'Waxwing', 'Orpington', 'Warbler', 'Canary', 'Macaw', 'Quail', 'Cardinal', 'Falcon', 'Egyptian', 'Starling', 'Gamefowl', 'Australorp', 'Polish', 'Pelican']


In [124]:
bird_replacements = {
    'Rhode Island' : 'Chicken',
    'Silkie' : 'Chicken',
    'Guinea' : 'Guinea Fowl',
    'African' : 'African Grey', 
    'Barred Rock' : 'Chicken',
    'Muscovy' : 'Muscovy Duck',
    'Song Bird' : 'Songbird',
    'Leghorn' : 'Chicken', 
    'Other Bird' : 'Other', 
    'Black' : 'Grackle',
    'Orpington' : 'Chicken',
    'Egyptian' : 'Other',
    'Gamefowl' : 'Other',
    'Australorp' : 'Chicken', 
    'Polish' : 'Chicken',
    'American' : 'Chicken',
    'Wyandottes' : 'Chicken',
    'Quaker' : 'Quaker Parrot'
}

In [125]:
# Replacements

aacIn = replace_breeds(aacIn, "Bird", bird_replacements)

Checking for 'Rhode Island' in 'PrimaryBreed'. Found: 10 times.
Replaced 10 occurrences of 'Rhode Island' with 'Chicken'.

Checking for 'Silkie' in 'PrimaryBreed'. Found: 6 times.
Replaced 6 occurrences of 'Silkie' with 'Chicken'.

Checking for 'Guinea' in 'PrimaryBreed'. Found: 3 times.
Replaced 3 occurrences of 'Guinea' with 'Guinea Fowl'.

Checking for 'African' in 'PrimaryBreed'. Found: 1 times.
Replaced 1 occurrences of 'African' with 'African Grey'.

Checking for 'Barred Rock' in 'PrimaryBreed'. Found: 5 times.
Replaced 5 occurrences of 'Barred Rock' with 'Chicken'.

Checking for 'Muscovy' in 'PrimaryBreed'. Found: 7 times.
Replaced 7 occurrences of 'Muscovy' with 'Muscovy Duck'.

Checking for 'Song Bird' in 'PrimaryBreed'. Found: 1 times.
Replaced 1 occurrences of 'Song Bird' with 'Songbird'.

Checking for 'Leghorn' in 'PrimaryBreed'. Found: 2 times.
Replaced 2 occurrences of 'Leghorn' with 'Chicken'.

Checking for 'Other Bird' in 'PrimaryBreed'. Found: 1 times.
Replaced 1 occur

In [126]:
# Evaluating Wildlife

print(aacIn.loc[aacIn['AnimalType'] == 'Wildlife', 'PrimaryBreed'].unique().tolist())

['Bat', 'Raccoon', 'Skunk', 'Opossum', 'Fox', 'Armadillo', 'Squirrel', 'Turtle', 'Rat', 'Coyote', 'Tortoise', 'Cottontail', 'Cold Water', 'Snake', 'Lizard', 'Mouse', 'Tarantula', 'Frog', 'Ringtail', 'Bobcat', 'Otter', 'Deer', 'Prairie Dog']


In [127]:
wild_replacements = {
    'Cottontail' : 'Cottontail Rabbit',
    'Cold Water' : 'Fish',
    'Mex Free-Tail' : 'Bat'
}

In [128]:
# Replace

aacIn = replace_breeds(aacIn, "Wildlife", wild_replacements)

Checking for 'Cottontail' in 'PrimaryBreed'. Found: 5 times.
Replaced 5 occurrences of 'Cottontail' with 'Cottontail Rabbit'.

Checking for 'Cold Water' in 'PrimaryBreed'. Found: 35 times.
Replaced 35 occurrences of 'Cold Water' with 'Fish'.

Checking for 'Mex Free-Tail' in 'PrimaryBreed'. Found: 0 times.

After replacements, the unique breeds in 'PrimaryBreed' for Wildlife are now: 23

Total replacements made: 40


In [129]:
# Looking through rabbits

print(aacIn.loc[aacIn['AnimalType'] == 'Rabbit', 'PrimaryBreed'].unique().tolist())

['Rabbit Sh', 'Californian', 'Rex', 'English Spot', 'Lionhead', 'Silver', 'Dutch', 'Jersey Wooly', 'Lop-English', 'American', 'Lop-Holland', 'Rabbit Lh', 'American Sable', 'New Zealand Wht', 'Lop-Mini', 'Hotot', 'Cinnamon', 'Angora-French', 'Netherlnd Dwarf', 'Rex-Mini', 'Checkered Giant', 'Havana', 'Polish', 'Harlequin', 'Angora-English', 'Lop-Amer Fuzzy', 'Flemish Giant', 'Belgian Hare', 'Britannia Petit', 'Himalayan', 'Rhinelander', 'Beveren', 'Angora-Satin', 'Dwarf Hotot', 'Florida White', 'Lop-French']


In [130]:
bun_replacements = {
    'Rabbit Sh' : 'Shorthair', 
    'Lop-English' : 'English Lop',
    'Lop-Holland' : 'Holland Lop',
    'Rabbit Lh' : 'Longhair', 
    'New Zealand Wht' : 'New Zealand White',
    'Lop-Mini' : 'Mini Lop',
    'Angora-French' : 'French Angora',
    'Netherlnd Dwarf' : 'Netherland Dwarf',
    'Rex-Mini' : 'Mini Rex', 
    'Angora-English' : 'English Angora',
    'Lop-Amer Fuzzy' : 'American Fuzzy Lop',
    'Angora-Satin' : 'Satin Angora',
    'Lop-French' : 'French Lop',
}

In [131]:
# Replacing

aacIn = replace_breeds(aacIn, "Rabbit", bun_replacements)

Checking for 'Rabbit Sh' in 'PrimaryBreed'. Found: 660 times.
Replaced 660 occurrences of 'Rabbit Sh' with 'Shorthair'.

Checking for 'Lop-English' in 'PrimaryBreed'. Found: 15 times.
Replaced 15 occurrences of 'Lop-English' with 'English Lop'.

Checking for 'Lop-Holland' in 'PrimaryBreed'. Found: 28 times.
Replaced 28 occurrences of 'Lop-Holland' with 'Holland Lop'.

Checking for 'Rabbit Lh' in 'PrimaryBreed'. Found: 30 times.
Replaced 30 occurrences of 'Rabbit Lh' with 'Longhair'.

Checking for 'New Zealand Wht' in 'PrimaryBreed'. Found: 21 times.
Replaced 21 occurrences of 'New Zealand Wht' with 'New Zealand White'.

Checking for 'Lop-Mini' in 'PrimaryBreed'. Found: 15 times.
Replaced 15 occurrences of 'Lop-Mini' with 'Mini Lop'.

Checking for 'Angora-French' in 'PrimaryBreed'. Found: 5 times.
Replaced 5 occurrences of 'Angora-French' with 'French Angora'.

Checking for 'Netherlnd Dwarf' in 'PrimaryBreed'. Found: 13 times.
Replaced 13 occurrences of 'Netherlnd Dwarf' with 'Netherlan

In [132]:
# Almost done--Livestock...

print(aacIn.loc[aacIn['AnimalType'] == 'Livestock', 'PrimaryBreed'].unique().tolist())

['Pig', 'Goat', 'Potbelly Pig', 'Sheep', 'Pygmy', 'Emu']


In [133]:
live_replacements = {
    'Potbelly Pig' : 'Pig',
    'Pygmy' : 'Goat'
}

In [134]:
# Just a couple replacements this time

aacIn = replace_breeds(aacIn, "Livestock", live_replacements)

Checking for 'Potbelly Pig' in 'PrimaryBreed'. Found: 4 times.
Replaced 4 occurrences of 'Potbelly Pig' with 'Pig'.

Checking for 'Pygmy' in 'PrimaryBreed'. Found: 1 times.
Replaced 1 occurrences of 'Pygmy' with 'Goat'.

After replacements, the unique breeds in 'PrimaryBreed' for Livestock are now: 4

Total replacements made: 5


In [135]:
# Finally, Other...

print(aacIn.loc[aacIn['AnimalType'] == 'Other', 'PrimaryBreed'].unique().tolist())

['Ferret', 'Guinea Pig', 'Hamster', 'Sugar Glider', 'Gerbil', 'Hedgehog', 'Chinchilla-Amer', 'Chinchilla-Stnd', 'Chinchilla', 'Tropical', 'Hermit Crab', 'Snake', 'Lizard']


In [136]:
other_replacements = {
    'Chinchilla-Amer' : 'Chinchilla',
    'Chinchilla-Stnd' : 'Chinchilla',
    'Tropical' : 'Fish',
    'Python' : 'Snake',
    'Bearded Dragon' : 'Lizard'
}

In [137]:
# Last replacements...

aacIn = replace_breeds(aacIn, "Other", other_replacements)

Checking for 'Chinchilla-Amer' in 'PrimaryBreed'. Found: 2 times.
Replaced 2 occurrences of 'Chinchilla-Amer' with 'Chinchilla'.

Checking for 'Chinchilla-Stnd' in 'PrimaryBreed'. Found: 3 times.
Replaced 3 occurrences of 'Chinchilla-Stnd' with 'Chinchilla'.

Checking for 'Tropical' in 'PrimaryBreed'. Found: 8 times.
Replaced 8 occurrences of 'Tropical' with 'Fish'.

Checking for 'Python' in 'PrimaryBreed'. Found: 0 times.

Checking for 'Bearded Dragon' in 'PrimaryBreed'. Found: 0 times.

After replacements, the unique breeds in 'PrimaryBreed' for Other are now: 11

Total replacements made: 13


I think that's finally it, for aacIn's Breed categories. Now we check our work and move on!

##### Last Steps--Checking Our Work

In [138]:
print(aacIn['PrimaryBreed'].loc[aacIn['AnimalType'] == 'Dog'].value_counts())

PrimaryBreed
American Pit Bull Terrier    15221
Labrador Retriever           13530
Chihuahua                    10921
German Shepherd               6997
Australian Cattle Dog         3421
                             ...  
Dutch Sheepdog                   1
Wolf Hybrid                      1
Borzoi                           1
Scottish Deerhound               1
Tosa                             1
Name: count, Length: 196, dtype: int64


In [139]:
print(aacIn['PrimaryBreed'].loc[aacIn['AnimalType'] == 'Cat'].value_counts())

PrimaryBreed
Domestic Shorthair         57573
Domestic Medium Hair        5454
Domestic Longhair           2479
Siamese                     2044
Snowshoe                     285
American Shorthair           284
Maine Coon                   162
Manx                         144
Russian Blue                 139
Ragdoll                       76
Himalayan                     63
Persian                       43
Bengal                        31
Abyssinian                    28
Turkish Angora                21
Balinese                      20
American Curl Shorthair       17
British Shorthair             16
Tonkinese                     13
Devon Rex                     11
Japanese Bobtail              11
Bombay                        10
Burmese                        9
Turkish Van                    9
Cymric                         8
Scottish Fold                  6
Cornish Rex                    6
Havana Brown                   6
Oriental Shorthair             5
Munchkin Shorthair            

In [140]:
print(aacIn['PrimaryBreed'].loc[aacIn['AnimalType'] == 'Bird'].value_counts())

PrimaryBreed
Chicken          378
Parakeet         104
Duck              76
Pigeon            43
Dove              35
Cockatiel         30
Grackle           24
Finch             15
Peafowl           14
Parrot            12
Quaker Parrot     10
Hawk              10
Sparrow           10
Bantam            10
Lovebird           8
Owl                8
Muscovy Duck       7
Turkey             7
Conure             7
Vulture            7
Mockingbird        7
Heron              5
Other              4
Cockatoo           4
Budgerigar         4
Bluebird           4
Guinea Fowl        3
Pheasant           3
Macaw              3
Goose              2
Waxwing            2
Crow               2
Catbird            2
Quail              2
Starling           1
Falcon             1
Cardinal           1
Canary             1
Warbler            1
Chickadee          1
Lark               1
Whimbrel           1
Wren               1
Songbird           1
African Grey       1
Nuthatch           1
Bunting            1


In [141]:
print(aacIn['PrimaryBreed'].loc[aacIn['AnimalType'] == 'Rabbit'].value_counts())

PrimaryBreed
Shorthair             660
Californian            71
Lionhead               46
Rex                    37
Longhair               30
Holland Lop            28
English Spot           22
New Zealand White      21
Havana                 15
Mini Lop               15
English Lop            15
Netherland Dwarf       13
American               12
Dutch                  12
Hotot                   7
Flemish Giant           7
English Angora          6
Mini Rex                6
Himalayan               6
French Angora           5
American Fuzzy Lop      5
Jersey Wooly            4
Cinnamon                4
American Sable          3
Harlequin               3
Polish                  3
Belgian Hare            2
Checkered Giant         2
Satin Angora            2
Dwarf Hotot             2
Florida White           2
Silver                  1
Britannia Petit         1
Rhinelander             1
Beveren                 1
French Lop              1
Name: count, dtype: int64


In [142]:
print(aacIn['PrimaryBreed'].loc[aacIn['AnimalType'] == 'Livestock'].value_counts())

PrimaryBreed
Pig      25
Goat      5
Emu       2
Sheep     1
Name: count, dtype: int64


In [143]:
print(aacIn['PrimaryBreed'].loc[aacIn['AnimalType'] == 'Other'].value_counts())

PrimaryBreed
Guinea Pig      705
Hamster          80
Ferret           78
Chinchilla       15
Gerbil            9
Hedgehog          8
Fish              8
Sugar Glider      4
Snake             4
Lizard            2
Hermit Crab       1
Name: count, dtype: int64


I've noticed something in this process: some wildlife, birds, or other animals have an IsMix entry of True. Since these animals likely aren't actually mixed-breeds, I'm going to remove it from them.

In [144]:
# Starting with birds

aacIn.loc[aacIn['AnimalType'] == 'Bird', 'IsMix'] = False

# Checking that it worked right

print(aacIn['IsMix'].loc[aacIn['AnimalType'] == 'Bird'].value_counts())

IsMix
False    876
Name: count, dtype: int64


In [145]:
# Now onto Livestock

aacIn.loc[aacIn['AnimalType'] == 'Livestock', 'IsMix'] = False

# Wildlife

aacIn.loc[aacIn['AnimalType'] == 'Wildlife', 'IsMix'] = False

# And Other species

aacIn.loc[aacIn['AnimalType'] == 'Other', 'IsMix'] = False

In [146]:
# Rabbits and Cats may be suspect...let's check out how many of those animal types are mixes

aacIn.loc[(aacIn['AnimalType'] == 'Cat') & (aacIn['IsMix'] == True), :].shape[0]

41313

That's a pretty large number of cats...I'm going to leave them be, for the time being.

In [147]:
aacIn.loc[(aacIn['AnimalType'] == 'Rabbit') & (aacIn['IsMix'] == True), :].shape[0]

589

That's about half of the total rabbits. Since the breeds for rabbits were so specific and numerous, I'm going to let them keep their mixes.

In [148]:
# Check our work one last time

aacIn.sample(5)

,AnimalID,Name,IntakeDT,IntakeType,IntakeCondition,AnimalType,IntakeSex,IntakeAge,Breed,Color,AgeYears,AgeCategory,IsMix,PrimaryBreed
88384,A779669,nan,2018-09-04 13:02:00,Public Assist,Normal,Bird,Intact Male,1 year,Chicken,Red/White,1,Young,False,Chicken
64960,A748953,Sambo,2017-05-09 18:14:00,Public Assist,Normal,Dog,Neutered Male,2 years,Labrador Retriever,Yellow,2,Young,False,Labrador Retriever
10554,A678987,nan,2014-05-14 23:57:00,Stray,Nursing,Cat,Intact Female,3 days,Domestic Medium Hair Mix,Tortie,0,Infant,True,Domestic Medium Hair
10860,A679381,Cola,2014-05-20 11:46:00,Public Assist,Normal,Dog,Intact Male,1 year,Chihuahua Shorthair/Dachshund,Tan/White,1,Young,True,Chihuahua
31096,A705150,Jax,2015-06-14 01:03:00,Public Assist,Normal,Dog,Intact Male,2 months,Staffordshire Mix,Brown/Black,0,Infant,True,Staffordshire Bull Terrier


#### Outcomes

##### Preliminary Cleaning (Again)

In [149]:
# Start with the same steps we used earlier

# Clean the breeds column up first by making sure there are no case-related misspellings

aacOut['Breed'] = aacOut['Breed'].str.lower().str.strip()

print(aacOut['Breed'].value_counts())

Breed
domestic shorthair mix                34027
domestic shorthair                    23537
pit bull mix                          10130
labrador retriever mix                 8784
chihuahua shorthair mix                6910
                                      ...  
labrador retriever/saluki                 1
cairn terrier/cardigan welsh corgi        1
siamese/munchkin shorthair                1
smooth fox terrier/beagle                 1
dalmatian/english springer spaniel        1
Name: count, Length: 3002, dtype: int64


In [150]:
# And add those capitals back

aacOut['Breed'] = aacOut['Breed'].str.title()
print(aacOut['Breed'].value_counts())

Breed
Domestic Shorthair Mix                34027
Domestic Shorthair                    23537
Pit Bull Mix                          10130
Labrador Retriever Mix                 8784
Chihuahua Shorthair Mix                6910
                                      ...  
Labrador Retriever/Saluki                 1
Cairn Terrier/Cardigan Welsh Corgi        1
Siamese/Munchkin Shorthair                1
Smooth Fox Terrier/Beagle                 1
Dalmatian/English Springer Spaniel        1
Name: count, Length: 3002, dtype: int64


3002 is almost exactly the number of breeds we had in the Intake dataset at this point--perfect.

In [151]:
# Apply "parse_breeds" to this dataset as well

aacOut[['IsMix', 'PrimaryBreed', 'SecondaryBreed']] = aacOut['Breed'].apply(parse_breeds)

In [152]:
# Check it

aacOut[['AnimalType', 'Breed', 'IsMix', 'PrimaryBreed', 'SecondaryBreed']].head(15)

,AnimalType,Breed,IsMix,PrimaryBreed,SecondaryBreed
0,Cat,Domestic Shorthair Mix,True,Domestic Shorthair,None
1,Cat,Domestic Shorthair Mix,True,Domestic Shorthair,None
2,Dog,Chihuahua Shorthair Mix,True,Chihuahua Shorthair,None
3,Wildlife,Raccoon,False,Raccoon,None
4,Dog,Anatol Shepherd/Labrador Retriever,True,Anatol Shepherd,Labrador Retriever
5,Cat,Domestic Shorthair Mix,True,Domestic Shorthair,None
6,Dog,Chihuahua Shorthair Mix,True,Chihuahua Shorthair,None
7,Dog,American Foxhound/Labrador Retriever,True,American Foxhound,Labrador Retriever
8,Cat,Domestic Shorthair,False,Domestic Shorthair,None
9,Cat,Domestic Shorthair,False,Domestic Shorthair,None


In [153]:
# Looking at the new categories we have now

print(aacOut['PrimaryBreed'].value_counts())
print("Primary breed null values: " + str(aacOut['PrimaryBreed'].isnull().sum()))
print()
print(aacOut['SecondaryBreed'].value_counts())
print('Secondary breed null values: ' + str(aacOut['SecondaryBreed'].isnull().sum()))

PrimaryBreed
Domestic Shorthair              57582
Pit Bull                        14433
Labrador Retriever              13501
Chihuahua Shorthair             10089
German Shepherd                  6968
                                ...  
Nuthatch                            1
Britannia Petit                     1
Spanish Mastiff                     1
Grand Basset Griffon Vendeen        1
Sheep                               1
Name: count, Length: 386, dtype: int64
Primary breed null values: 0

SecondaryBreed
Labrador Retriever     2287
Pit Bull               1185
Chihuahua Shorthair    1080
German Shepherd        1034
Dachshund               729
                       ... 
Pygmy                     1
Angora                    1
Tibetan Spaniel           1
Irish Terrier             1
Californian               1
Name: count, Length: 211, dtype: int64
Secondary breed null values: 157483


Again, almost exactly the same numbers as the Intakes dataset, which bodes well

In [154]:
# Removing the SecondaryBreed column--too many nulls

aacOut.drop(columns = 'SecondaryBreed', inplace = True)

aacOut.head()

,AnimalID,Name,OutcomeDT,BirthDate,OutcomeType,OutcomeSubtype,AnimalType,OutcomeSex,OutcomeAge,Breed,Color,AgeYears,AgeCategory,IsMix,PrimaryBreed
0,A882831,Hamilton,2023-07-01 18:12:00,2023-03-25,Adoption,NaN,Cat,Neutered Male,3 months,Domestic Shorthair Mix,Black/White,0,Infant,True,Domestic Shorthair
1,A794011,Chunk,2019-05-08 18:20:00,2017-05-02,Return to Owner,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,2,Young,True,Domestic Shorthair
2,A776359,Gizmo,2018-07-18 16:02:00,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,1,Young,True,Chihuahua Shorthair
3,A821648,nan,2020-08-16 11:38:00,2019-08-16,Euthanasia,NaN,Wildlife,Unknown,1 year,Raccoon,Gray,1,Young,False,Raccoon
4,A720371,Moose,2016-02-13 17:59:00,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,0,Infant,True,Anatol Shepherd


##### Manually Checking and Replacing Breeds (Again)

This time (thankfully) this should be easier, since I already have dictionaries to apply (though I may still need to double-check that there are no new messy values in this dataset)

In [155]:
# Replacements: dogs

aacOut = replace_breeds(aacOut, "Dog", dog_replacements)

Checking for 'Black' in 'PrimaryBreed'. Found: 232 times.
Replaced 232 occurrences of 'Black' with 'Other'.

Checking for 'Doberman Pinsch' in 'PrimaryBreed'. Found: 533 times.
Replaced 533 occurrences of 'Doberman Pinsch' with 'Doberman Pinscher'.

Checking for 'Chinese Sharpei' in 'PrimaryBreed'. Found: 316 times.
Replaced 316 occurrences of 'Chinese Sharpei' with 'Chinese Shar-Pei'.

Checking for 'Dachshund Wirehair' in 'PrimaryBreed'. Found: 223 times.
Replaced 223 occurrences of 'Dachshund Wirehair' with 'Dachshund'.

Checking for 'Chihuahua Shorthair' in 'PrimaryBreed'. Found: 10089 times.
Replaced 10089 occurrences of 'Chihuahua Shorthair' with 'Chihuahua'.

Checking for 'Flat Coat Retriever' in 'PrimaryBreed'. Found: 159 times.
Replaced 159 occurrences of 'Flat Coat Retriever' with 'Flat-Coated Retriever'.

Checking for 'Dachshund Longhair' in 'PrimaryBreed'. Found: 258 times.
Replaced 258 occurrences of 'Dachshund Longhair' with 'Dachshund'.

Checking for 'Chihuahua Longhair' 

In [156]:
# Go through the remainder for any sneaky weird values

pprint(aacOut['PrimaryBreed'][aacOut['AnimalType'] == 'Dog'].unique().tolist())

['Chihuahua',
 'Anatolian Shepherd',
 'American Foxhound',
 'Border Collie',
 'American Pit Bull Terrier',
 'Labrador Retriever',
 'Black Mouth Cur',
 'Weimaraner',
 'Yorkshire Terrier',
 'Jack Russell Terrier',
 'Great Pyrenees',
 'Australian Cattle Dog',
 'Beagle',
 'German Shepherd',
 'Dogo Argentino',
 'Cairn Terrier',
 'Rough Collie',
 'Rat Terrier',
 'Doberman Pinscher',
 'American Bulldog',
 'American Staffordshire Terrier',
 'Standard Poodle',
 'Miniature Poodle',
 'Dachshund',
 'Bloodhound',
 'Standard Schnauzer',
 'Pug',
 'Australian Shepherd',
 'Siberian Husky',
 'Australian Kelpie',
 'Miniature Schnauzer',
 'Miniature Pinscher',
 'Boxer',
 'Catahoula Leopard Dog',
 'Staffordshire Bull Terrier',
 'Bluetick Coonhound',
 'Border Terrier',
 'Flat-Coated Retriever',
 'Mastiff',
 'American Eskimo',
 'Boston Terrier',
 'Vizsla',
 'Shiba Inu',
 'Soft Coated Wheaten Terrier',
 'German Shorthaired Pointer',
 'Rottweiler',
 'Akita',
 'English Bulldog',
 'Harrier',
 'French Bulldog',
 

In [157]:
aacOut = replace_breeds(aacOut, "Cat", cat_replacements)

Checking for 'Oriental Sh' in 'PrimaryBreed'. Found: 5 times.
Replaced 5 occurrences of 'Oriental Sh' with 'Oriental Shorthair'.

Checking for 'Angora' in 'PrimaryBreed'. Found: 13 times.
Replaced 13 occurrences of 'Angora' with 'Turkish Angora'.

Checking for 'Colorpoint' in 'PrimaryBreed'. Found: 3 times.
Replaced 3 occurrences of 'Colorpoint' with 'Colorpoint Shorthair'.

Checking for 'Rex' in 'PrimaryBreed'. Found: 1 times.
Replaced 1 occurrences of 'Rex' with 'Cornish Rex'.

After replacements, the unique breeds in 'PrimaryBreed' for Cat are now: 42

Total replacements made: 22


In [158]:
pprint(aacOut['PrimaryBreed'][aacOut['AnimalType'] == 'Cat'].unique().tolist())

['Domestic Shorthair',
 'Domestic Medium Hair',
 'Siamese',
 'Russian Blue',
 'Persian',
 'Domestic Longhair',
 'Snowshoe',
 'Bombay',
 'American Shorthair',
 'Abyssinian',
 'Munchkin Shorthair',
 'Bengal',
 'Manx',
 'Turkish Angora',
 'Maine Coon',
 'Himalayan',
 'Japanese Bobtail',
 'Tonkinese',
 'Devon Rex',
 'Ragdoll',
 'British Shorthair',
 'Balinese',
 'Cymric',
 'Scottish Fold',
 'Norwegian Forest Cat',
 'Cornish Rex',
 'American Curl Shorthair',
 'Burmese',
 'Oriental Shorthair',
 'Birman',
 'Colorpoint Shorthair',
 'Turkish Van',
 'American Wirehair',
 'Exotic Shorthair',
 'Savannah',
 'Sphynx',
 'Havana Brown',
 'Chartreux',
 'Pixiebob Shorthair',
 'Ocicat',
 'Munchkin Longhair',
 'Javanese']


In [159]:
aacOut = replace_breeds(aacOut, "Bird", bird_replacements)

Checking for 'Rhode Island' in 'PrimaryBreed'. Found: 10 times.
Replaced 10 occurrences of 'Rhode Island' with 'Chicken'.

Checking for 'Silkie' in 'PrimaryBreed'. Found: 6 times.
Replaced 6 occurrences of 'Silkie' with 'Chicken'.

Checking for 'Guinea' in 'PrimaryBreed'. Found: 3 times.
Replaced 3 occurrences of 'Guinea' with 'Guinea Fowl'.

Checking for 'African' in 'PrimaryBreed'. Found: 1 times.
Replaced 1 occurrences of 'African' with 'African Grey'.

Checking for 'Barred Rock' in 'PrimaryBreed'. Found: 5 times.
Replaced 5 occurrences of 'Barred Rock' with 'Chicken'.

Checking for 'Muscovy' in 'PrimaryBreed'. Found: 6 times.
Replaced 6 occurrences of 'Muscovy' with 'Muscovy Duck'.

Checking for 'Song Bird' in 'PrimaryBreed'. Found: 1 times.
Replaced 1 occurrences of 'Song Bird' with 'Songbird'.

Checking for 'Leghorn' in 'PrimaryBreed'. Found: 2 times.
Replaced 2 occurrences of 'Leghorn' with 'Chicken'.

Checking for 'Other Bird' in 'PrimaryBreed'. Found: 1 times.
Replaced 1 occur

In [160]:
pprint(aacOut['PrimaryBreed'][aacOut['AnimalType'] == 'Bird'].unique().tolist())

['Parakeet',
 'Chicken',
 'Quaker Parrot',
 'Dove',
 'Peafowl',
 'Waxwing',
 'Duck',
 'Pigeon',
 'Hawk',
 'Vulture',
 'Heron',
 'Goose',
 'Parrot',
 'Bantam',
 'Grackle',
 'Canary',
 'Falcon',
 'Lovebird',
 'Conure',
 'Owl',
 'Budgerigar',
 'Guinea Fowl',
 'Muscovy Duck',
 'Cockatiel',
 'Chickadee',
 'Macaw',
 'Finch',
 'Pheasant',
 'Other',
 'Sparrow',
 'Mockingbird',
 'Bluebird',
 'Cardinal',
 'Bunting',
 'Turkey',
 'Starling',
 'Crow',
 'Quail',
 'Pelican',
 'Nuthatch',
 'Cockatoo',
 'Warbler',
 'Lark',
 'African Grey',
 'Songbird',
 'Wren',
 'Catbird',
 'Whimbrel']


In [161]:
aacOut = replace_breeds(aacOut, "Wildlife", wild_replacements)

Checking for 'Cottontail' in 'PrimaryBreed'. Found: 5 times.
Replaced 5 occurrences of 'Cottontail' with 'Cottontail Rabbit'.

Checking for 'Cold Water' in 'PrimaryBreed'. Found: 35 times.
Replaced 35 occurrences of 'Cold Water' with 'Fish'.

Checking for 'Mex Free-Tail' in 'PrimaryBreed'. Found: 0 times.

After replacements, the unique breeds in 'PrimaryBreed' for Wildlife are now: 23

Total replacements made: 40


In [162]:
pprint(aacOut['PrimaryBreed'][aacOut['AnimalType'] == 'Wildlife'].unique().tolist())

['Raccoon',
 'Opossum',
 'Bat',
 'Lizard',
 'Fox',
 'Snake',
 'Squirrel',
 'Skunk',
 'Ringtail',
 'Rat',
 'Turtle',
 'Mouse',
 'Fish',
 'Tortoise',
 'Coyote',
 'Armadillo',
 'Tarantula',
 'Cottontail Rabbit',
 'Deer',
 'Otter',
 'Prairie Dog',
 'Bobcat',
 'Frog']


In [163]:
aacOut = replace_breeds(aacOut, "Rabbit", bun_replacements)

Checking for 'Rabbit Sh' in 'PrimaryBreed'. Found: 661 times.
Replaced 661 occurrences of 'Rabbit Sh' with 'Shorthair'.

Checking for 'Lop-English' in 'PrimaryBreed'. Found: 15 times.
Replaced 15 occurrences of 'Lop-English' with 'English Lop'.

Checking for 'Lop-Holland' in 'PrimaryBreed'. Found: 28 times.
Replaced 28 occurrences of 'Lop-Holland' with 'Holland Lop'.

Checking for 'Rabbit Lh' in 'PrimaryBreed'. Found: 31 times.
Replaced 31 occurrences of 'Rabbit Lh' with 'Longhair'.

Checking for 'New Zealand Wht' in 'PrimaryBreed'. Found: 21 times.
Replaced 21 occurrences of 'New Zealand Wht' with 'New Zealand White'.

Checking for 'Lop-Mini' in 'PrimaryBreed'. Found: 15 times.
Replaced 15 occurrences of 'Lop-Mini' with 'Mini Lop'.

Checking for 'Angora-French' in 'PrimaryBreed'. Found: 5 times.
Replaced 5 occurrences of 'Angora-French' with 'French Angora'.

Checking for 'Netherlnd Dwarf' in 'PrimaryBreed'. Found: 13 times.
Replaced 13 occurrences of 'Netherlnd Dwarf' with 'Netherlan

In [164]:
pprint(aacOut['PrimaryBreed'][aacOut['AnimalType'] == 'Rabbit'].unique().tolist())

['Polish',
 'Shorthair',
 'Californian',
 'Longhair',
 'Lionhead',
 'Holland Lop',
 'Dutch',
 'French Angora',
 'English Spot',
 'Satin Angora',
 'English Lop',
 'Rex',
 'Rhinelander',
 'Havana',
 'New Zealand White',
 'Netherland Dwarf',
 'Hotot',
 'Cinnamon',
 'English Angora',
 'American',
 'American Fuzzy Lop',
 'Mini Lop',
 'Checkered Giant',
 'American Sable',
 'Himalayan',
 'Flemish Giant',
 'French Lop',
 'Harlequin',
 'Mini Rex',
 'Florida White',
 'Jersey Wooly',
 'Silver',
 'Britannia Petit',
 'Beveren',
 'Dwarf Hotot',
 'Belgian Hare']


In [165]:
aacOut = replace_breeds(aacOut, "Livestock", live_replacements)

Checking for 'Potbelly Pig' in 'PrimaryBreed'. Found: 4 times.
Replaced 4 occurrences of 'Potbelly Pig' with 'Pig'.

Checking for 'Pygmy' in 'PrimaryBreed'. Found: 1 times.
Replaced 1 occurrences of 'Pygmy' with 'Goat'.

After replacements, the unique breeds in 'PrimaryBreed' for Livestock are now: 5

Total replacements made: 5


In [166]:
pprint(aacOut['PrimaryBreed'][aacOut['AnimalType'] == 'Livestock'].unique().tolist())

['Pig', 'Goat', 'Emu', 'Miniature', 'Sheep']


In [167]:
# We do actually have an issue here: what is "Miniature"? Could be a pig, a horse, a goat...
# for us since I have no way of knowing, it will be "Other"

live_replacements_add = {
    'Miniature' : 'Other'
}

aacOut = replace_breeds(aacOut, "Livestock", live_replacements_add)

print()
print(aacOut['PrimaryBreed'][aacOut['AnimalType'] == 'Livestock'].unique().tolist())

Checking for 'Miniature' in 'PrimaryBreed'. Found: 1 times.
Replaced 1 occurrences of 'Miniature' with 'Other'.

After replacements, the unique breeds in 'PrimaryBreed' for Livestock are now: 5

Total replacements made: 1

['Pig', 'Goat', 'Emu', 'Other', 'Sheep']


In [168]:
aacOut = replace_breeds(aacOut, "Other", other_replacements)

Checking for 'Chinchilla-Amer' in 'PrimaryBreed'. Found: 2 times.
Replaced 2 occurrences of 'Chinchilla-Amer' with 'Chinchilla'.

Checking for 'Chinchilla-Stnd' in 'PrimaryBreed'. Found: 3 times.
Replaced 3 occurrences of 'Chinchilla-Stnd' with 'Chinchilla'.

Checking for 'Tropical' in 'PrimaryBreed'. Found: 8 times.
Replaced 8 occurrences of 'Tropical' with 'Fish'.

Checking for 'Python' in 'PrimaryBreed'. Found: 0 times.

Checking for 'Bearded Dragon' in 'PrimaryBreed'. Found: 0 times.

After replacements, the unique breeds in 'PrimaryBreed' for Other are now: 11

Total replacements made: 13


In [169]:
pprint(aacOut['PrimaryBreed'][aacOut['AnimalType'] == 'Other'].unique().tolist())

['Guinea Pig',
 'Ferret',
 'Chinchilla',
 'Hamster',
 'Hedgehog',
 'Snake',
 'Gerbil',
 'Fish',
 'Lizard',
 'Hermit Crab',
 'Sugar Glider']


Breeds replaced! Let's move on to our final steps again!

##### Final Steps (Again)

In [170]:
# Check our work

print(aacOut['PrimaryBreed'].loc[aacOut['AnimalType'] == 'Dog'].value_counts())

PrimaryBreed
American Pit Bull Terrier    15138
Labrador Retriever           13501
Chihuahua                    10943
German Shepherd               6968
Australian Cattle Dog         3416
                             ...  
Scottish Deerhound               1
Borzoi                           1
Spanish Water Dog                1
Wolf Hybrid                      1
Bouvier des Flandres             1
Name: count, Length: 196, dtype: int64


In [171]:
print(aacOut['PrimaryBreed'].loc[aacOut['AnimalType'] == 'Cat'].value_counts())

PrimaryBreed
Domestic Shorthair         57582
Domestic Medium Hair        5478
Domestic Longhair           2477
Siamese                     2044
Snowshoe                     288
American Shorthair           284
Maine Coon                   166
Manx                         144
Russian Blue                 140
Ragdoll                       76
Himalayan                     63
Persian                       44
Bengal                        31
Abyssinian                    28
Turkish Angora                21
Balinese                      20
American Curl Shorthair       17
British Shorthair             15
Tonkinese                     13
Japanese Bobtail              11
Devon Rex                     11
Bombay                        10
Turkish Van                    9
Burmese                        9
Cymric                         8
Havana Brown                   6
Cornish Rex                    6
Pixiebob Shorthair             5
Munchkin Shorthair             5
Scottish Fold                 

In [172]:
print(aacOut['PrimaryBreed'].loc[aacOut['AnimalType'] == 'Wildlife'].value_counts())

PrimaryBreed
Bat                  4102
Raccoon              1358
Opossum               389
Skunk                 281
Fox                   205
Squirrel              163
Rat                   141
Snake                  46
Turtle                 43
Lizard                 38
Coyote                 38
Fish                   35
Mouse                  29
Tortoise               27
Armadillo              13
Tarantula               6
Cottontail Rabbit       5
Ringtail                3
Deer                    3
Otter                   2
Prairie Dog             1
Bobcat                  1
Frog                    1
Name: count, dtype: int64


In [173]:
print(aacOut['PrimaryBreed'].loc[aacOut['AnimalType'] == 'Rabbit'].value_counts())

PrimaryBreed
Shorthair             661
Californian            71
Lionhead               46
Rex                    37
Longhair               31
Holland Lop            28
English Spot           22
New Zealand White      21
Mini Lop               15
English Lop            15
Havana                 15
Netherland Dwarf       13
Dutch                  12
American               12
Flemish Giant           7
Hotot                   7
English Angora          6
Himalayan               6
Mini Rex                6
American Fuzzy Lop      5
French Angora           5
Cinnamon                4
Jersey Wooly            4
Harlequin               3
Polish                  3
American Sable          3
Checkered Giant         2
Florida White           2
Satin Angora            2
Dwarf Hotot             2
Belgian Hare            2
French Lop              1
Rhinelander             1
Silver                  1
Britannia Petit         1
Beveren                 1
Name: count, dtype: int64


In [174]:
print(aacOut['PrimaryBreed'].loc[aacOut['AnimalType'] == 'Bird'].value_counts())

PrimaryBreed
Chicken          377
Parakeet         104
Duck              76
Pigeon            43
Dove              35
Cockatiel         30
Grackle           24
Finch             15
Peafowl           14
Parrot            12
Quaker Parrot     10
Sparrow           10
Hawk              10
Bantam            10
Lovebird           8
Owl                8
Turkey             7
Vulture            7
Conure             7
Mockingbird        7
Muscovy Duck       6
Heron              5
Cockatoo           4
Budgerigar         4
Bluebird           4
Other              4
Pheasant           3
Guinea Fowl        3
Macaw              3
Catbird            2
Waxwing            2
Goose              2
Quail              2
Crow               2
Lark               1
Wren               1
African Grey       1
Songbird           1
Warbler            1
Chickadee          1
Nuthatch           1
Pelican            1
Starling           1
Bunting            1
Cardinal           1
Falcon             1
Canary             1


In [175]:
print(aacOut['PrimaryBreed'].loc[aacOut['AnimalType'] == 'Livestock'].value_counts())

PrimaryBreed
Pig      25
Goat      5
Emu       2
Other     1
Sheep     1
Name: count, dtype: int64


In [176]:
print(aacOut['PrimaryBreed'].loc[aacOut['AnimalType'] == 'Other'].value_counts())

PrimaryBreed
Guinea Pig      697
Hamster          81
Ferret           78
Chinchilla       15
Gerbil            9
Hedgehog          8
Fish              8
Sugar Glider      4
Snake             3
Lizard            2
Hermit Crab       1
Name: count, dtype: int64


In [177]:
# Now we fix the IsMix column

aacOut.loc[aacOut['AnimalType'] == 'Bird', 'IsMix'] = False  # Removing it for Birds...
aacOut.loc[aacOut['AnimalType'] == 'Livestock', 'IsMix'] = False  # For Livestock...
aacOut.loc[aacOut['AnimalType'] == 'Wildlife', 'IsMix'] = False  # For Wildlife...
aacOut.loc[aacOut['AnimalType'] == 'Other', 'IsMix'] = False  # And for Others

In [178]:
# Check them one last time

# Birds

print(aacOut['IsMix'].loc[aacOut['AnimalType'] == 'Bird'].value_counts())

IsMix
False    874
Name: count, dtype: int64


In [179]:
# Wildlife

print(aacOut['IsMix'].loc[aacOut['AnimalType'] == 'Wildlife'].value_counts())

IsMix
False    6930
Name: count, dtype: int64


In [180]:
# Others

print(aacOut['IsMix'].loc[aacOut['AnimalType'] == 'Other'].value_counts())

IsMix
False    906
Name: count, dtype: int64


In [181]:
aacOut.sample(5)

,AnimalID,Name,OutcomeDT,BirthDate,OutcomeType,OutcomeSubtype,AnimalType,OutcomeSex,OutcomeAge,Breed,Color,AgeYears,AgeCategory,IsMix,PrimaryBreed
77949,A685235,nan,2014-08-04 15:07:00,2013-10-04,Euthanasia,Suffering,Cat,Intact Male,9 months,Domestic Shorthair Mix,Flame Point,1,Infant,True,Domestic Shorthair
43933,A807123,nan,2019-10-25 19:40:00,2019-06-21,Adoption,NaN,Dog,Spayed Female,4 months,German Shepherd,Black/Cream,0,Infant,False,German Shepherd
149234,A746535,Old Red,2017-04-12 16:37:00,2009-04-04,Transfer,Partner,Dog,Intact Male,8 years,German Shepherd Mix,Red,8,Senior,True,German Shepherd
61598,A841272,Hermione,2021-09-18 09:46:00,2021-07-14,Adoption,NaN,Dog,Spayed Female,2 months,Australian Cattle Dog Mix,White/Tan,0,Infant,True,Australian Cattle Dog
44019,A750937,Perpetua,2017-06-09 00:00:00,2017-05-11,Transfer,Partner,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Orange Tabby,0,Infant,True,Domestic Shorthair


And that's all, finally, for the breeds! (It took a very long time.) Onto the last columns that need to be cleaned: Color!

### Color

#### Intakes

In [182]:
print(aacIn['Color'].value_counts())

Color
Black/White                 17928
Black                       14619
Brown Tabby                 10657
Brown                        7140
White                        6120
                            ...  
Red Merle/Tan                   1
Brown Brindle/Liver Tick        1
Green/Silver                    1
Black Smoke/Brown Tabby         1
Ruddy/Black                     1
Name: count, Length: 660, dtype: int64


660 is also too many categories to be useful for modeling or graphing. To solve this, I think I need to separate colors which are mixed with "/", much as I did for breeds, and save only the primary color. I can also later consolidate colors with just a few entries into an "Other" category.

In [183]:
def process_colors(color):
    """
    Split color combinations and keep only the first printed;
    Leave "Black/White" or "White/Black" as "Black/White".

    Args:
        color (str): Color, as a string.

    Returns:
        color (str): "Black/White" or the primary color of the string, minus "/".
    """
    if pd.isna(color):
        return None  # Missing values
    if color in ('Black/White', 'White/Black'):
        return 'Black/White'  # Keep these entries, as they refer to a common color scheme
    if '/' in color:
        return color.split('/')[0]  # Reduce color entries to the first color listed
    return color  # Return the first color

In [184]:
aacIn['ColorClean'] = aacIn['Color'].apply(process_colors)

print(aacIn['ColorClean'].value_counts())

ColorClean
Black           24071
Black/White     22751
Brown           17478
White           17243
Brown Tabby     16276
                ...  
Black Tiger         9
Orange Tiger        6
Ruddy               3
Cream Tiger         2
Unknown             1
Name: count, Length: 61, dtype: int64


That alone reduced our color count to 61, versus the previous 660 we had. Now to standardize things such as "Grey"/"Gray", "White/Black"/"Black/White", "Tiger"/"Tabby", etc.

In [185]:
pprint(aacIn['ColorClean'].unique())

array(['White', 'Orange', 'Blue', 'Black/White', 'Brown Tabby', 'Black',
       'Buff', 'Silver', 'Tortie', 'Tan', 'Orange Tabby', 'Brown', 'Red',
       'Tricolor', 'Apricot', 'Gray', 'Brown Brindle', 'Cream', 'Calico',
       'Black Tabby', 'Blue Tabby', 'Sable', 'Fawn', 'Torbie',
       'Chocolate', 'Blue Merle', 'Yellow', 'Gold', 'Cream Tabby',
       'Black Brindle', 'Black Smoke', 'Red Merle', 'Tortie Point',
       'Seal Point', 'Brown Merle', 'Liver', 'Lynx Point', 'Lilac Point',
       'Blue Tick', 'Brown Tiger', 'Blue Cream', 'Flame Point',
       'Blue Point', 'Calico Point', 'Chocolate Point', 'Red Tick',
       'Gray Tabby', 'Green', 'Silver Tabby', 'Yellow Brindle',
       'Blue Tiger', 'Blue Smoke', 'Ruddy', 'Black Tiger', 'Pink',
       'Agouti', 'Liver Tick', 'Silver Lynx Point', 'Orange Tiger',
       'Cream Tiger', 'Unknown'], dtype=object)


There are some that are still overly specific, so I may make a condensed column with just the most basic color (e.g. "Brown Tabby" -> "Brown") eventually.

In [186]:
color_remaps = {
    "Tortie" : "Tortoiseshell",
    "Tortie Point" : "Tortoiseshell Point",
    "Orange Tiger" : "Orange Tabby", # Some exploring shows there are no dogs in this group
    "Cream Tiger" : "Cream Tabby" # Same as above
}

# Brown, Blue, and Black Tiger are complicated, because they are correct if referring to dogs, but not to cats
    # (and all three of these are applied to both currently)

color_remap_not_dog = {
    "Brown Tiger" : "Brown Tabby",
    "Blue Tiger" : "Blue Tabby", 
    "Black Tiger" : "Black Tabby"
}

In [187]:
# For these, I don't need a full function the way I did for Breeds

# First the overall replacements

for old_val, new_val in color_remaps.items():
    aacIn.loc[(aacIn['ColorClean'] == old_val), 'ColorClean'] = new_val

# Then the cat-specific ones--only replace if the AnimalType refers to a cat

for old_value, new_value in color_remap_not_dog.items():
    aacIn.loc[(aacIn['AnimalType'] == "Cat") & (aacIn['ColorClean'] == old_value), 'ColorClean'] = new_value

# Check it worked...

print(aacIn['AnimalType'].loc[(aacIn['ColorClean'] == 'Brown Tiger')].value_counts())

AnimalType
Dog    37
Name: count, dtype: int64


In [188]:
pprint(aacIn['ColorClean'].unique())
print()
print(aacIn['ColorClean'].nunique())

array(['White', 'Orange', 'Blue', 'Black/White', 'Brown Tabby', 'Black',
       'Buff', 'Silver', 'Tortoiseshell', 'Tan', 'Orange Tabby', 'Brown',
       'Red', 'Tricolor', 'Apricot', 'Gray', 'Brown Brindle', 'Cream',
       'Calico', 'Black Tabby', 'Blue Tabby', 'Sable', 'Fawn', 'Torbie',
       'Chocolate', 'Blue Merle', 'Yellow', 'Gold', 'Cream Tabby',
       'Black Brindle', 'Black Smoke', 'Red Merle', 'Tortoiseshell Point',
       'Seal Point', 'Brown Merle', 'Liver', 'Lynx Point', 'Lilac Point',
       'Blue Tick', 'Blue Cream', 'Flame Point', 'Blue Point',
       'Calico Point', 'Chocolate Point', 'Red Tick', 'Gray Tabby',
       'Green', 'Silver Tabby', 'Yellow Brindle', 'Blue Tiger',
       'Blue Smoke', 'Ruddy', 'Pink', 'Agouti', 'Liver Tick',
       'Silver Lynx Point', 'Brown Tiger', 'Black Tiger', 'Unknown'],
      dtype=object)

59


It appears to have worked! So now, instead of 600 colors and combinations of colors, we have 69 categories, which is much easier to work with.

#### Outcomes

For the Outcomes dataset, all I need to do is essentially the same thing I did for Intakes, and then double-check that there are not any new anomalies that need fixing.

In [189]:
# Our function is already defined, so we just apply it

aacOut['ColorClean'] = aacOut['Color'].apply(process_colors)

In [190]:
# Overall replacements, using the dicts we created for aacIn

for old_val, new_val in color_remaps.items():
    aacOut.loc[(aacOut['ColorClean'] == old_val), 'ColorClean'] = new_val

# Cat-specific replacements

for old_value, new_value in color_remap_not_dog.items():
    aacOut.loc[(aacOut['AnimalType'] == "Cat") & (aacOut['ColorClean'] == old_value), 'ColorClean'] = new_value

# Check it worked...

print(aacOut['AnimalType'].loc[(aacOut['ColorClean'] == 'Brown Tiger')].value_counts())

AnimalType
Dog    37
Name: count, dtype: int64


In [191]:
# Check it worked and there are no further issues

print(aacOut['ColorClean'].unique())

['Black/White' 'Brown Tabby' 'White' 'Gray' 'Buff' 'Orange Tabby' 'Brown'
 'Blue Tabby' 'Black' 'Blue' 'Red' 'Torbie' 'Lilac Point' 'Calico'
 'Tricolor' 'Chocolate' 'Brown Brindle' 'Tan' 'Cream Tabby' 'Blue Tick'
 'Fawn' 'Tortoiseshell' 'Sable' 'Cream' 'Blue Merle' 'Silver Lynx Point'
 'Brown Merle' 'Silver' 'Apricot' 'Green' 'Tortoiseshell Point'
 'Seal Point' 'Lynx Point' 'Black Brindle' 'Yellow' 'Yellow Brindle'
 'Chocolate Point' 'Blue Smoke' 'Silver Tabby' 'Gray Tabby' 'Orange'
 'Brown Tiger' 'Gold' 'Flame Point' 'Black Tabby' 'Black Smoke'
 'Calico Point' 'Blue Cream' 'Agouti' 'Red Merle' 'Blue Point' 'Red Tick'
 'Liver' 'Liver Tick' 'Black Tiger' 'Pink' 'Blue Tiger' 'Unknown' 'Ruddy']


In [192]:
# Just making sure "Green" is assigned to the appropriate species, for my own peace of mind

print(aacOut[['AnimalType', 'PrimaryBreed']].loc[(aacOut['ColorClean'] == 'Green')].value_counts())

AnimalType  PrimaryBreed 
Bird        Parakeet         33
Wildlife    Turtle           27
Bird        Parrot            9
            Quaker Parrot     9
Wildlife    Lizard            8
            Tortoise          5
Bird        Conure            4
            Lovebird          4
            Peafowl           2
            Budgerigar        1
            Cockatiel         1
            Macaw             1
Wildlife    Snake             1
Name: count, dtype: int64


And that's it for color! Relatively painless, after Breeds...

## Final Steps

Checking dtypes one more time

What I should see is
* String/Object: AnimalID, Name, IntakeType, IntakeCondition, AnimalType, IntakeSex, IntakeAge (that original column), Breed, Color, AgeCategory, PrimaryBreed, ColorClean, OutcomeType, OutcomeSubtype, OutcomeSex, OutcomeAge (again, that original column)
* Datetime: IntakeDT, OutcomeDT, BirthDate
* Integer: AgeYears
* Boolean: IsMix

In [193]:
print(aacIn.dtypes)
print() 
print(aacOut.dtypes)

AnimalID                   object
Name                       object
IntakeDT           datetime64[ns]
IntakeType                 object
IntakeCondition            object
AnimalType                 object
IntakeSex                  object
IntakeAge                  object
Breed                      object
Color                      object
AgeYears                    Int64
AgeCategory                object
IsMix                        bool
PrimaryBreed               object
ColorClean                 object
dtype: object

AnimalID                  object
Name                      object
OutcomeDT         datetime64[ns]
BirthDate         datetime64[ns]
OutcomeType               object
OutcomeSubtype            object
AnimalType                object
OutcomeSex                object
OutcomeAge                object
Breed                     object
Color                     object
AgeYears                   Int64
AgeCategory               object
IsMix                       bool
PrimaryBreed 

### Revisiting Nulls

In [194]:
# Revisiting missing values

print(aacIn.isnull().sum())
print()
print(aacOut.isnull().sum())

AnimalID           0
Name               0
IntakeDT           0
IntakeType         0
IntakeCondition    0
AnimalType         0
IntakeSex          2
IntakeAge          1
Breed              0
Color              0
AgeYears           1
AgeCategory        0
IsMix              0
PrimaryBreed       0
ColorClean         0
dtype: int64

AnimalID              0
Name                  0
OutcomeDT             0
BirthDate             0
OutcomeType          46
OutcomeSubtype    93762
AnimalType            0
OutcomeSex            2
OutcomeAge            9
Breed                 0
Color                 0
AgeYears              9
AgeCategory           0
IsMix                 0
PrimaryBreed          0
ColorClean            0
dtype: int64


In [195]:
# IntakeSex & OutcomeSex--fill with "Unknown"

aacIn['IntakeSex'] = aacIn['IntakeSex'].fillna('Unknown')
aacOut['OutcomeSex'] = aacOut['OutcomeSex'].fillna('Unknown')

print(aacIn['IntakeSex'].value_counts())
print()
print(aacOut['OutcomeSex'].value_counts())

IntakeSex
Intact Male      58813
Intact Female    56512
Neutered Male    24190
Spayed Female    20280
Unknown          13462
Name: count, dtype: int64

OutcomeSex
Neutered Male    60681
Spayed Female    55062
Intact Male      22167
Intact Female    21731
Unknown          13411
Name: count, dtype: int64


In [196]:
# IntakeAge & OutcomeAge: I'm going to employ the same method again, and set them as "Unknown"

aacIn['IntakeAge'] = aacIn['IntakeAge'].fillna('Unknown')
aacOut['OutcomeAge'] = aacOut['OutcomeAge'].fillna('Unknown')

print(aacIn['IntakeAge'].value_counts())
print()
print(aacOut['OutcomeAge'].value_counts())

IntakeAge
1 year       28202
2 years      27952
1 month      18148
2 months     10409
3 years       9581
4 weeks       6295
4 years       5827
5 years       5177
3 weeks       5101
3 months      4900
4 months      4773
5 months      4368
2 weeks       3650
6 years       3421
6 months      3325
7 years       3001
8 years       2829
7 months      2585
9 months      2487
10 years      2288
8 months      2025
9 years       1670
1 week        1469
1 weeks       1413
10 months     1383
0 years       1247
11 months     1137
12 years      1116
1 day         1009
11 years       923
3 days         920
2 days         768
13 years       715
6 days         532
14 years       523
4 days         497
15 years       451
5 weeks        444
5 days         263
16 years       192
17 years       104
18 years        62
19 years        29
20 years        26
22 years         6
1 years          6
25 years         2
24 years         1
21 years         1
23 years         1
Unknown          1
30 years         1
28

In [197]:
# AgeYears: fill with the median

aacIn['AgeYears'] = aacIn['AgeYears'].fillna(aacIn['AgeYears'].median())
aacOut['AgeYears'] = aacOut['AgeYears'].fillna(aacOut['AgeYears'].median())

print(aacIn['AgeYears'].isna().sum())
print()
print(aacOut['AgeYears'].isna().sum())

0

0


In [198]:
# OutcomeType

# Remind myself what the current categories are...

print(aacOut['OutcomeType'].value_counts())

OutcomeType
Adoption           84214
Transfer           48527
Return to Owner    26865
Euthanasia         11638
Died                1663
Missing               99
Name: count, dtype: int64


In [199]:
# I think we're going to add those nulls to "Missing" as, after all, the Outcome is missing.
# (It's obviously not a direct correlation, but it's close enough)

aacOut['OutcomeType'] = aacOut['OutcomeType'].fillna('Missing')

print(aacOut['OutcomeType'].value_counts())

OutcomeType
Adoption           84214
Transfer           48527
Return to Owner    26865
Euthanasia         11638
Died                1663
Missing              145
Name: count, dtype: int64


In [200]:
# OutcomeSubtype

# Remind myself again, what categories we have

print(aacOut['OutcomeSubtype'].value_counts())

OutcomeSubtype
Partner                  40247
Foster                   17851
Rabies Risk               4891
Suffering                 4123
Shelter Neuter Return     4078
Scratched Person          3208
Out of State               931
In Kennel                  868
Aggressive                 612
Offsite                    512
In Foster                  416
Medical                    353
At Vet                     352
In Field                   242
Behavior                   176
Enroute                    112
Court/Investigation        100
Emergency                   64
Underage                    37
In Surgery                  33
Customer S                  20
Pet Resource Center         20
Possible Theft              16
Barn                        16
In State                    12
Name: count, dtype: int64


In [201]:
# As much as I would like to have these details around,
    # I think there are not enough of them to justify keeping the column,
    # as it is not particularly helpful to the scope of this project;
    # OutcomeType is enough.

aacOut = aacOut.drop(columns = 'OutcomeSubtype')

print(aacOut.columns)

Index(['AnimalID', 'Name', 'OutcomeDT', 'BirthDate', 'OutcomeType',
       'AnimalType', 'OutcomeSex', 'OutcomeAge', 'Breed', 'Color', 'AgeYears',
       'AgeCategory', 'IsMix', 'PrimaryBreed', 'ColorClean'],
      dtype='object')


### Joining the dataframes

In [202]:
# Prep datasets for merging by organizing them by AnimalID and datetimes

aacIn_sorted = aacIn.sort_values(['IntakeDT', 'AnimalID'])
aacOut_sorted = aacOut.sort_values(['OutcomeDT', 'AnimalID'])

aac = pd.merge_asof(
    aacIn_sorted,
    aacOut_sorted,
    by='AnimalID',
    left_on='IntakeDT',
    right_on='OutcomeDT',
    direction='forward',
    allow_exact_matches=True
).dropna(subset=['OutcomeDT'])  # Ensures only matched outcomes remain

In [203]:
# Check columns

print(aac.columns)

Index(['AnimalID', 'Name_x', 'IntakeDT', 'IntakeType', 'IntakeCondition',
       'AnimalType_x', 'IntakeSex', 'IntakeAge', 'Breed_x', 'Color_x',
       'AgeYears_x', 'AgeCategory_x', 'IsMix_x', 'PrimaryBreed_x',
       'ColorClean_x', 'Name_y', 'OutcomeDT', 'BirthDate', 'OutcomeType',
       'AnimalType_y', 'OutcomeSex', 'OutcomeAge', 'Breed_y', 'Color_y',
       'AgeYears_y', 'AgeCategory_y', 'IsMix_y', 'PrimaryBreed_y',
       'ColorClean_y'],
      dtype='object')


Let's investigate how closely matched each column (x vs y) is.

In [204]:
# Checking how closely columns match

name_match = (aac['Name_x'] == aac['Name_y']).mean()
print(f"Row-wise Name match rate: {name_match:.2f}")

type_match = (aac['AnimalType_x'] == aac['AnimalType_y']).mean()
print(f"Row-wise AnimalType match rate: {type_match:.2f}")

breed_match = (aac['Breed_x'] == aac['Breed_y']).mean()
print(f"Row-wise Breed match rate: {breed_match:.2f}")

color_match = (aac['Color_x'] == aac['Color_y']).mean()
print(f"Row-wise Color match rate: {color_match:.2f}")

age_yr_match = (aac['AgeYears_x'] <= aac['AgeYears_y']).mean()
print(f"Row-wise AgeYears (Intake =< Outcome) rate: {age_yr_match:.2f}")

age_cat_match = (aac['AgeCategory_x'] == aac['AgeCategory_y']).mean()
print(f"Row-wise AgeCategory match rate: {age_cat_match:.2f}")

mix_match = (aac['IsMix_x'] == aac['IsMix_y']).mean()
print(f"Row-wise IsMix match rate: {mix_match:.2f}")

breed1_match = (aac['PrimaryBreed_x'] == aac['PrimaryBreed_y']).mean()
print(f"Row-wise PrimaryBreed match rate: {breed1_match:.2f}")

color2_match = (aac['ColorClean_x'] == aac['ColorClean_y']).mean()
print(f"Row-wise ColorClean match rate: {color2_match:.2f}")

Row-wise Name match rate: 1.00
Row-wise AnimalType match rate: 1.00
Row-wise Breed match rate: 1.00
Row-wise Color match rate: 1.00
Row-wise AgeYears (Intake =< Outcome) rate: 1.00
Row-wise AgeCategory match rate: 0.99
Row-wise IsMix match rate: 1.00
Row-wise PrimaryBreed match rate: 1.00
Row-wise ColorClean match rate: 1.00


Almost all of them match perfectly, which is encouraging!

AgeYears seems to be correct, but AgeCategory is slightly off; however, since Intake Ages were likely guesses to begin with, I can probably just ignore the age-related _x columns (including IntakeAge, AgeYears_x, and AgeCategory_x).

In [205]:
# Let's just drop those redundant age columns now

aac = aac.drop(columns=['IntakeAge', 'AgeYears_x', 'AgeCategory_x'])
print(aac.columns)

Index(['AnimalID', 'Name_x', 'IntakeDT', 'IntakeType', 'IntakeCondition',
       'AnimalType_x', 'IntakeSex', 'Breed_x', 'Color_x', 'IsMix_x',
       'PrimaryBreed_x', 'ColorClean_x', 'Name_y', 'OutcomeDT', 'BirthDate',
       'OutcomeType', 'AnimalType_y', 'OutcomeSex', 'OutcomeAge', 'Breed_y',
       'Color_y', 'AgeYears_y', 'AgeCategory_y', 'IsMix_y', 'PrimaryBreed_y',
       'ColorClean_y'],
      dtype='object')


In [206]:
# Check numerical columns for outliers

print(aac.describe())

                            IntakeDT                      OutcomeDT  \
count                         171489                         171489   
mean   2018-11-08 17:46:25.272991232  2018-11-29 20:07:44.658607616   
min              2013-10-01 07:51:00            2013-10-01 10:39:00   
25%              2016-02-07 13:00:00            2016-02-24 09:56:00   
50%              2018-07-18 13:23:00            2018-08-10 16:38:00   
75%              2021-07-27 17:13:00            2021-08-19 00:00:00   
max              2025-04-09 07:26:00            2025-04-09 14:50:00   
std                              NaN                            NaN   

                           BirthDate  AgeYears_y  
count                         171489    171489.0  
mean   2016-10-27 07:54:14.881654272    1.924893  
min              1991-09-22 00:00:00         0.0  
25%              2014-03-26 00:00:00         0.0  
50%              2016-11-13 00:00:00         1.0  
75%              2019-12-19 00:00:00         2.0  
max

* Okay, most things look okay, but we seem to have a couple anomalies.
* We also have a BirthDate in 2028, which would be an impressive feat of time travel for an animal
* A BirthDate in 1991 is feasible, but I will check it in case--1991 to 2013 (at the earliest) would be quite an old animal
* And again, an age of 30 is feasible, but warrants a little bit of suspicion

In [207]:
# I also noticed some BirthDates coming after the animals OutcomeDate, which is incorrect

aac.loc[(aac['BirthDate'] > aac['OutcomeDT']), ['AnimalID', 'BirthDate', 'OutcomeDT', 'OutcomeAge']]

,AnimalID,BirthDate,OutcomeDT,OutcomeAge
17408,A751749,2014-12-12,2014-09-10 17:29:00,0 years
28695,A702326,2015-08-29,2015-05-24 17:01:00,0 years
29597,A703416,2015-05-29,2015-05-26 16:58:00,0 years
32007,A753893,2016-07-12,2015-07-02 11:06:00,1 years
32556,A706929,2015-07-06,2015-07-05 14:46:00,0 years
43840,A754280,2016-07-12,2016-04-11 00:00:00,0 years
49607,A725472,2020-04-29,2016-07-24 15:54:00,3 years
55394,A736114,2016-10-28,2016-10-04 15:13:00,0 years
56431,A737397,2016-11-15,2016-11-05 18:16:00,0 years
62128,A745085,2017-10-11,2017-03-13 18:11:00,0 years


In [208]:
# One of these, at least, is an obvious error (BirthDate in 2028). Assuming the animal's AgeYears entry is correct, I correct the value

aac.loc[171830, 'BirthDate'] = pd.to_datetime('2021-05-01')

# There is still the matter of the other 30, however

In [209]:
# I am uncertain what to do with the rest, but given that I'm not sure what pieces are incorrect, and it is only 30 rows, I'm going to drop them

aac = aac[aac['BirthDate'] <= aac['OutcomeDT']]

print(aac.describe())

                            IntakeDT                      OutcomeDT  \
count                         171459                         171459   
mean   2018-11-08 17:42:41.083407616  2018-11-29 20:07:43.260371456   
min              2013-10-01 07:51:00            2013-10-01 10:39:00   
25%              2016-02-07 12:25:00            2016-02-24 08:07:30   
50%              2018-07-18 13:17:00            2018-08-10 16:15:00   
75%              2021-07-27 21:01:00            2021-08-19 04:02:30   
max              2025-04-09 07:26:00            2025-04-09 14:50:00   
std                              NaN                            NaN   

                           BirthDate  AgeYears_y  
count                         171459    171459.0  
mean   2016-10-27 03:05:13.243399424    1.925142  
min              1991-09-22 00:00:00         0.0  
25%              2014-03-26 00:00:00         0.0  
50%              2016-11-13 00:00:00         1.0  
75%              2019-12-19 00:00:00         2.0  
max

In [210]:
# Let's look at the rest of BirthDate and AgeYears together, since they directly correspond
# Calculating age from BirthDate--do they match?

aac['BirthAge'] = (aac['OutcomeDT'] - aac['BirthDate']).dt.days / 365.25

# Check the difference between the two

aac['AgeDiff'] = aac['BirthAge'] - aac['AgeYears_y']

# Summary of age differences
print(aac['AgeDiff'].describe())

# Check for outliers
mismatches = aac[aac['AgeDiff'].abs() > 1.5]  # Difference larger than 1 year
mismatches[['AnimalID', 'AnimalType_x', 'BirthDate', 'OutcomeDT', 'AgeYears_y', 'BirthAge', 'AgeDiff']]

count    171459.0
mean     0.164101
std      0.230384
min          -1.0
25%      0.013005
50%      0.103354
75%      0.260096
max           7.0
Name: AgeDiff, dtype: Float64


,AnimalID,AnimalType_x,BirthDate,OutcomeDT,AgeYears_y,BirthAge,AgeDiff
170972,A923553,Dog,2022-02-04,2025-02-21 11:57:00,1,3.047228,2.047228
172838,A927996,Cat,2017-04-05,2025-04-05 15:32:00,1,8.000000,7.0


In [211]:
# Only 2 significant mismatches--whew. For these, I will manually adjust the AgeYears_y value

aac.loc[170972, 'AgeYears_y'] = 3
aac.loc[172838, 'AgeYears_y'] = 8

In [212]:
# Check that it worked

mismatches = aac[aac['AgeDiff'].abs() > 1.5]  # The same rows we looked at before
mismatches[['AnimalID', 'AnimalType_x', 'BirthDate', 'OutcomeDT', 'AgeYears_y', 'BirthAge', 'AgeDiff']]

,AnimalID,AnimalType_x,BirthDate,OutcomeDT,AgeYears_y,BirthAge,AgeDiff
170972,A923553,Dog,2022-02-04,2025-02-21 11:57:00,3,3.047228,2.047228
172838,A927996,Cat,2017-04-05,2025-04-05 15:32:00,8,8.000000,7.0


In [213]:
# So now the ages and birthdates should align, it appears that 1991 BirthDate does agree with the animal's age
# Let's look at some very early BirthDates and very old animals

aac.loc[(aac['BirthDate'] < pd.to_datetime('1995-01-01')), ['AnimalID', 'AnimalType_x', 'PrimaryBreed_x', 'BirthDate', 'OutcomeDT', 'OutcomeAge']]

,AnimalID,AnimalType_x,PrimaryBreed_x,BirthDate,OutcomeDT,OutcomeAge
3465,A214991,Cat,Domestic Shorthair,1994-03-01,2013-12-14 13:28:00,19 years
5198,A559240,Dog,Miniature Poodle,1994-10-21,2014-01-25 12:02:00,19 years
5218,A671488,Cat,Domestic Shorthair,1994-01-25,2014-09-17 11:18:00,20 years
6096,A672841,Cat,Domestic Medium Hair,1994-02-17,2014-02-20 14:52:00,20 years
12914,A682010,Cat,Domestic Shorthair,1992-06-25,2014-06-25 13:26:00,22 years
19199,A689957,Cat,Domestic Medium Hair,1994-10-11,2014-10-11 17:58:00,20 years
20462,A261770,Cat,Domestic Shorthair,1994-11-07,2014-11-07 15:51:00,20 years
21128,A692485,Cat,Domestic Shorthair,1994-11-21,2014-11-25 16:04:00,20 years
22067,A693678,Cat,Domestic Shorthair,1994-12-12,2014-12-13 16:38:00,20 years
37279,A712996,Cat,Siamese,1993-10-03,2015-10-01 13:46:00,22 years


Cats can get pretty old, so that looks fine to me. One dog making it to 19 isn't shocking either, and birds and tortoises both live long lives. These look correct!

In [214]:
# Looking at animals with old listed ages

aac.loc[(aac['AgeYears_y'] > 20), ['AnimalID', 'AnimalType_x', 'PrimaryBreed_x', 'IsMix_x', 'BirthDate', 'OutcomeDT', 'AgeYears_y', 'OutcomeType']]

,AnimalID,AnimalType_x,PrimaryBreed_x,IsMix_x,BirthDate,OutcomeDT,AgeYears_y,OutcomeType
12914,A682010,Cat,Domestic Shorthair,True,1992-06-25,2014-06-25 13:26:00,22,Return to Owner
37279,A712996,Cat,Siamese,True,1993-10-03,2015-10-01 13:46:00,22,Return to Owner
45323,A723385,Cat,Domestic Shorthair,True,1994-04-03,2016-04-03 15:48:00,22,Return to Owner
58445,A740127,Bird,Cockatoo,False,1991-12-11,2016-12-16 00:00:00,25,Transfer
61872,A744732,Cat,Domestic Shorthair,True,1995-03-06,2017-03-06 17:09:00,22,Euthanasia
83791,A773881,Dog,German Shepherd,True,1994-06-07,2018-06-07 15:33:00,24,Return to Owner
96373,A790169,Wildlife,Bat,False,1998-03-05,2019-03-06 09:55:00,21,Euthanasia
119393,A821351,Cat,Domestic Medium Hair,False,1998-08-10,2020-08-10 13:08:00,22,Return to Owner
119961,A498745,Dog,Wolf Hybrid,True,1997-08-31,2020-08-31 12:36:00,23,Euthanasia
133057,A842878,Bird,Macaw,False,1991-09-22,2021-10-30 15:07:00,30,Transfer


Again, we have some ancient cats, some birds and tortoises, and then a couple of dogs. The dogs do seem odd to me, but since they are mixed breeds, I think it is feasible (if likely overestimated).

The numerical columns appear to be in order!

In [215]:
# Also checking for duplicates

print(aac.duplicated().sum())

0


In [216]:
# Sanity check time columns as well (do outcome dates actually come after intake dates?)

print((aac['IntakeDT'] > aac['OutcomeDT']).sum())

0


Now to remove unhelpful columns.

In [217]:
# First we list the columns

aac_cols = aac.columns
print(aac_cols)

Index(['AnimalID', 'Name_x', 'IntakeDT', 'IntakeType', 'IntakeCondition',
       'AnimalType_x', 'IntakeSex', 'Breed_x', 'Color_x', 'IsMix_x',
       'PrimaryBreed_x', 'ColorClean_x', 'Name_y', 'OutcomeDT', 'BirthDate',
       'OutcomeType', 'AnimalType_y', 'OutcomeSex', 'OutcomeAge', 'Breed_y',
       'Color_y', 'AgeYears_y', 'AgeCategory_y', 'IsMix_y', 'PrimaryBreed_y',
       'ColorClean_y', 'BirthAge', 'AgeDiff'],
      dtype='object')


In [218]:
full_clean_aac = aac.drop(columns = ['Name_x', 'AnimalType_x', 'Breed_x', 'Color_x', 'IsMix_x',
                                     'PrimaryBreed_x', 'ColorClean_x', 'BirthAge', 'AgeDiff'])

In [219]:
col_rename = {
    'Name_y' : 'Name',
    'AnimalType_y' : 'AnimalType',
    'Breed_y' : 'BreedOG',
    'Color_y' : 'ColorOG',
    'AgeYears_y' : 'AgeYears',
    'AgeCategory_y' : 'AgeCategory',
    'IsMix_y' : 'IsMix',
    'PrimaryBreed_y' : 'PrimaryBreed',
    'ColorClean_y' : 'ColorClean'
}

full_clean_aac.rename(columns = col_rename, inplace=True)
full_clean_aac.columns

Index(['AnimalID', 'IntakeDT', 'IntakeType', 'IntakeCondition', 'IntakeSex',
       'Name', 'OutcomeDT', 'BirthDate', 'OutcomeType', 'AnimalType',
       'OutcomeSex', 'OutcomeAge', 'BreedOG', 'ColorOG', 'AgeYears',
       'AgeCategory', 'IsMix', 'PrimaryBreed', 'ColorClean'],
      dtype='object')

In [220]:
# Reorder the remaining columns for clarity

full_aac_order = ['AnimalID', 'Name', 'AnimalType', 'BreedOG', 'PrimaryBreed', 'IsMix', 'ColorOG', 'ColorClean',
                  'BirthDate', 'OutcomeAge', 'AgeYears', 'AgeCategory', 'IntakeSex', 'IntakeType', 
                  'IntakeCondition', 'OutcomeType', 'OutcomeSex', 'IntakeDT', 'OutcomeDT']

full_clean_aac = full_clean_aac[full_aac_order + [col for col in full_clean_aac.columns if col not in full_aac_order]]

full_clean_aac.head()

,AnimalID,Name,AnimalType,BreedOG,PrimaryBreed,IsMix,ColorOG,ColorClean,BirthDate,OutcomeAge,AgeYears,AgeCategory,IntakeSex,IntakeType,IntakeCondition,OutcomeType,OutcomeSex,IntakeDT,OutcomeDT
0,A521520,Nina,Dog,Border Terrier/Border Collie,Border Terrier,True,White/Tan,White,2006-09-07,7 years,7,Adult,Spayed Female,Stray,Normal,Return to Owner,Spayed Female,2013-10-01 07:51:00,2013-10-01 15:39:00
1,A664235,nan,Cat,Domestic Shorthair Mix,Domestic Shorthair,True,Orange/White,Orange,2013-09-24,1 week,0,Infant,Unknown,Stray,Normal,Transfer,Unknown,2013-10-01 08:33:00,2013-10-01 10:39:00
2,A664236,nan,Cat,Domestic Shorthair Mix,Domestic Shorthair,True,Orange/White,Orange,2013-09-24,1 week,0,Infant,Unknown,Stray,Normal,Transfer,Unknown,2013-10-01 08:33:00,2013-10-01 10:44:00
3,A664237,nan,Cat,Domestic Shorthair Mix,Domestic Shorthair,True,Orange/White,Orange,2013-09-24,1 week,0,Infant,Unknown,Stray,Normal,Transfer,Unknown,2013-10-01 08:33:00,2013-10-01 10:44:00
4,A664233,Stevie,Dog,Pit Bull Mix,American Pit Bull Terrier,True,Blue/White,Blue,2010-09-30,3 years,3,Adult,Intact Female,Stray,Injured,Euthanasia,Intact Female,2013-10-01 08:53:00,2013-10-01 15:33:00


### Saving Final, Cleaned Dataframes

In [221]:
print(aacIn.columns)

print(aacOut.columns)

Index(['AnimalID', 'Name', 'IntakeDT', 'IntakeType', 'IntakeCondition',
       'AnimalType', 'IntakeSex', 'IntakeAge', 'Breed', 'Color', 'AgeYears',
       'AgeCategory', 'IsMix', 'PrimaryBreed', 'ColorClean'],
      dtype='object')
Index(['AnimalID', 'Name', 'OutcomeDT', 'BirthDate', 'OutcomeType',
       'AnimalType', 'OutcomeSex', 'OutcomeAge', 'Breed', 'Color', 'AgeYears',
       'AgeCategory', 'IsMix', 'PrimaryBreed', 'ColorClean'],
      dtype='object')


In [222]:
intakes = aacIn[['AnimalID', 'Name', 'IntakeDT', 'IntakeType', 'IntakeCondition', 'IntakeSex', 'AnimalType', 
                 'Breed', 'PrimaryBreed', 'IsMix', 'Color', 'ColorClean', 'IntakeAge', 'AgeYears', 'AgeCategory']]

outcomes = aacOut[['AnimalID', 'Name', 'OutcomeDT', 'OutcomeType', 'AnimalType', 'Breed', 'PrimaryBreed', 'IsMix',
                  'Color', 'ColorClean', 'OutcomeSex', 'BirthDate', 'OutcomeAge', 'AgeYears', 'AgeCategory']]

In [223]:
# Check intakes one last time

intakes.head()

,AnimalID,Name,IntakeDT,IntakeType,IntakeCondition,IntakeSex,AnimalType,Breed,PrimaryBreed,IsMix,Color,ColorClean,IntakeAge,AgeYears,AgeCategory
0,A521520,Nina,2013-10-01 07:51:00,Stray,Normal,Spayed Female,Dog,Border Terrier/Border Collie,Border Terrier,True,White/Tan,White,7 years,7,Adult
1,A664235,nan,2013-10-01 08:33:00,Stray,Normal,Unknown,Cat,Domestic Shorthair Mix,Domestic Shorthair,True,Orange/White,Orange,1 week,0,Infant
2,A664236,nan,2013-10-01 08:33:00,Stray,Normal,Unknown,Cat,Domestic Shorthair Mix,Domestic Shorthair,True,Orange/White,Orange,1 week,0,Infant
3,A664237,nan,2013-10-01 08:33:00,Stray,Normal,Unknown,Cat,Domestic Shorthair Mix,Domestic Shorthair,True,Orange/White,Orange,1 week,0,Infant
4,A664233,Stevie,2013-10-01 08:53:00,Stray,Injured,Intact Female,Dog,Pit Bull Mix,American Pit Bull Terrier,True,Blue/White,Blue,3 years,3,Adult


In [224]:
# As well as outcomes

outcomes.head()

,AnimalID,Name,OutcomeDT,OutcomeType,AnimalType,Breed,PrimaryBreed,IsMix,Color,ColorClean,OutcomeSex,BirthDate,OutcomeAge,AgeYears,AgeCategory
0,A882831,Hamilton,2023-07-01 18:12:00,Adoption,Cat,Domestic Shorthair Mix,Domestic Shorthair,True,Black/White,Black/White,Neutered Male,2023-03-25,3 months,0,Infant
1,A794011,Chunk,2019-05-08 18:20:00,Return to Owner,Cat,Domestic Shorthair Mix,Domestic Shorthair,True,Brown Tabby/White,Brown Tabby,Neutered Male,2017-05-02,2 years,2,Young
2,A776359,Gizmo,2018-07-18 16:02:00,Adoption,Dog,Chihuahua Shorthair Mix,Chihuahua,True,White/Brown,White,Neutered Male,2017-07-12,1 year,1,Young
3,A821648,nan,2020-08-16 11:38:00,Euthanasia,Wildlife,Raccoon,Raccoon,False,Gray,Gray,Unknown,2019-08-16,1 year,1,Young
4,A720371,Moose,2016-02-13 17:59:00,Adoption,Dog,Anatol Shepherd/Labrador Retriever,Anatolian Shepherd,True,Buff,Buff,Neutered Male,2015-10-08,4 months,0,Infant


In [225]:
# Final saves!

intakes.to_csv('../Data/Clean/intakes_clean.csv', index=False)
outcomes.to_csv('../Data/Clean/outcomes_clean.csv', index = False)
full_clean_aac.to_csv('../Data/Clean/joined_clean.csv', index = False)

# Concluding

## For EDA

### Features to Possibly Create or Alter:

* Fixed in shelter column
* Time in shelter
* Possibly split sex from fixed status
* Outcome success?
* Basic color categories (brown, blue, tan...)
* Simplify breeds

### Groupings to Evaluate:

* Outcome by Breed/Age/Sex
* Length of Stay by IntakeType/Condition/Breed/Age/Sex
* Outcome by Intake Condition
* Seasonal trends in intake and outcome types, Seasonal trends in adoptions